# Stroke Prediction Analysis - Statistical Hypothesis Testing

## 📋 Notebook Overview

This notebook performs comprehensive statistical hypothesis testing for the stroke prediction dataset, building on the data quality assessment from the previous notebook. We'll establish statistical evidence for risk factors through rigorous testing methodology.

### 🎯 Objectives

1. **Implement Missing Value Strategy**: Apply evidence-based imputation techniques
2. **Define Research Hypotheses**: Establish clear, testable hypotheses for each risk factor
3. **Test Statistical Assumptions**: Validate normality and other test prerequisites
4. **Perform Hypothesis Tests**: Apply appropriate statistical tests with proper corrections
5. **Quantify Effect Sizes**: Measure clinical significance beyond statistical significance
6. **Generate Evidence Base**: Create foundation for feature engineering and modeling

### 🔬 Scientific Methodology

We follow rigorous medical research standards:
- **Hypothesis Formation**: Clear null and alternative hypotheses
- **Assumption Testing**: Validate statistical test prerequisites
- **Appropriate Test Selection**: Choose tests based on data characteristics
- **Multiple Testing Correction**: Control family-wise error rate
- **Effect Size Analysis**: Clinical significance assessment
- **Medical Domain Validation**: Interpret findings in clinical context

### 📊 Expected Hypotheses

Based on medical literature, we expect to test:

1. **H₁ (Age)**: Older age is associated with higher stroke risk
2. **H₂ (Hypertension)**: Hypertension increases stroke risk
3. **H₃ (Heart Disease)**: Heart disease increases stroke risk
4. **H₄ (Glucose)**: Higher glucose levels increase stroke risk
5. **H₅ (BMI)**: BMI is associated with stroke risk
6. **H₆ (Smoking)**: Smoking status affects stroke risk
7. **H₇ (Gender)**: Gender influences stroke risk patterns
8. **H₈ (Social Factors)**: Marital status and work type affect stroke risk

### 🏥 Clinical Significance Standards

- **Odds Ratio**: >1.5 for clinical relevance
- **Effect Size**: Cohen's d >0.3 for practical significance
- **Age Difference**: >5 years for clinical meaning
- **Glucose Difference**: >20 mg/dL for clinical relevance
- **BMI Difference**: >2 kg/m² for clinical significance

## 🔧 Library Imports and Statistical Setup

Comprehensive setup for advanced statistical hypothesis testing with medical domain expertise.

In [4]:
# Core data processing libraries
import pandas as pd
import polars as pl
import numpy as np
from pathlib import Path

# Statistical testing libraries
from scipy import stats
from scipy.stats import (
    chi2_contingency, fisher_exact, ttest_ind, mannwhitneyu,
    shapiro, kstest, anderson, normaltest,
    pearsonr, spearmanr, pointbiserialr,
    levene, bartlett, f_oneway, kruskal
)

# Advanced statistical analysis
import statsmodels.api as sm
from statsmodels.stats.multitest import multipletests
from statsmodels.stats.contingency_tables import mcnemar
from statsmodels.stats.proportion import proportions_ztest
from statsmodels.stats.weightstats import ttest_ind as ttest_ind_sm

# Missing value imputation
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import LabelEncoder

# Visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.figure_factory as ff

# Utilities
import warnings
from typing import Dict, List, Tuple, Optional, Union
from datetime import datetime
import json

# Configure settings
warnings.filterwarnings('ignore')
plt.style.use('default')
sns.set_palette("husl")

# Configure display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pl.Config.set_tbl_rows(20)

print("✅ All libraries imported successfully!")
print(f"SciPy version: {stats.__version__ if hasattr(stats, '__version__') else 'Available'}")
print(f"Statsmodels available: {sm.__version__}")

✅ All libraries imported successfully!
SciPy version: Available
Statsmodels available: 0.14.4


## 📂 Data Loading and Missing Value Implementation

Loading the dataset from the previous analysis and implementing the evidence-based missing value strategy.

In [9]:
class StrokeHypothesisTestingPipeline:
    """
    Comprehensive pipeline for statistical hypothesis testing in stroke prediction
    """
    
    def __init__(self, show_progress: bool = True):
        self.show_progress = show_progress
        self.df_original = None
        self.df_cleaned = None
        self.missing_strategy_log = []
        self.test_results = {}
        self.effect_sizes = {}
        
    def load_data_from_quality_analysis(self, file_path: str) -> pl.DataFrame:
        """
        Load data with reference to quality analysis findings
        """
        if self.show_progress:
            print("="*80)
            print("📂 LOADING DATA FROM QUALITY ANALYSIS")
            print("="*80)
        
        # Try multiple possible paths
        possible_paths = [
            file_path,
            "healthcare-dataset-stroke-data.csv",
            "data/healthcare-dataset-stroke-data.csv",
            "../data/healthcare-dataset-stroke-data.csv"
        ]
        
        df = None
        loaded_path = None
        
        for path in possible_paths:
            try:
                if Path(path).exists():
                    df = pl.read_csv(
                        path,
                        infer_schema_length=10000,
                        try_parse_dates=True,
                        null_values=['', 'NULL', 'null', 'NA', 'N/A', 'nan', 'NaN'],
                        ignore_errors=False
                    )
                    loaded_path = path
                    break
            except Exception as e:
                continue
        
        if df is None:
            if self.show_progress:
                print("⚠️  Could not find original dataset, creating sample for demonstration...")
            
            # Create sample dataset for demonstration
            np.random.seed(42)
            sample_data = {
                'id': range(1, 5111),
                'gender': np.random.choice(['Male', 'Female', 'Other'], 5110, p=[0.45, 0.54, 0.01]),
                'age': np.random.normal(43, 22, 5110).clip(18, 95),
                'hypertension': np.random.choice([0, 1], 5110, p=[0.9, 0.1]),
                'heart_disease': np.random.choice([0, 1], 5110, p=[0.95, 0.05]),
                'ever_married': np.random.choice(['Yes', 'No'], 5110, p=[0.65, 0.35]),
                'work_type': np.random.choice(['Private', 'Self-employed', 'Govt_job', 'children', 'Never_worked'], 
                                            5110, p=[0.57, 0.16, 0.13, 0.12, 0.02]),
                'Residence_type': np.random.choice(['Urban', 'Rural'], 5110, p=[0.5, 0.5]),
                'avg_glucose_level': np.random.normal(106, 45, 5110).clip(55, 300),
                'bmi': np.random.normal(28.9, 7.5, 5110).clip(15, 50),
                'smoking_status': np.random.choice(['never smoked', 'formerly smoked', 'smokes', 'Unknown'], 
                                                 5110, p=[0.37, 0.17, 0.15, 0.31]),
                'stroke': np.random.choice([0, 1], 5110, p=[0.951, 0.049])
            }
            
            df = pl.DataFrame(sample_data)
            
            # Add some missing values to BMI (realistic pattern)
            bmi_array = df['bmi'].to_numpy()
            missing_indices = np.random.choice(len(bmi_array), size=int(0.039 * len(bmi_array)), replace=False)
            bmi_array[missing_indices] = None
            df = df.with_columns(pl.Series('bmi', bmi_array))
            
            loaded_path = "sample_dataset"
        
        if self.show_progress:
            print(f"✅ Dataset loaded successfully!")
            print(f"   Source: {loaded_path}")
            print(f"   Shape: {df.shape[0]:,} rows × {df.shape[1]} columns")
            
            # Quick quality overview
            missing_count = sum([df[col].null_count() for col in df.columns])
            print(f"   Missing values: {missing_count:,}")
            
            if 'stroke' in df.columns:
                stroke_rate = df['stroke'].mean() * 100
                print(f"   Stroke prevalence: {stroke_rate:.1f}%")
        
        self.df_original = df
        return df
    
    def implement_missing_value_strategy(self, df: pl.DataFrame) -> pl.DataFrame:
        """
        Implement evidence-based missing value imputation strategy
        """
        if self.show_progress:
            print("\n" + "="*80)
            print("🔧 IMPLEMENTING MISSING VALUE STRATEGY")
            print("="*80)
        
        df_cleaned = df.clone()
        
        # Analyze missing values first
        missing_analysis = {}
        for col in df.columns:
            missing_count = df[col].null_count()
            if missing_count > 0:
                missing_pct = (missing_count / df.height) * 100
                missing_analysis[col] = {
                    'count': missing_count,
                    'percentage': missing_pct
                }
        
        if self.show_progress:
            print(f"\n📊 Missing Values Overview:")
            if missing_analysis:
                for col, info in missing_analysis.items():
                    print(f"   • {col}: {info['count']} ({info['percentage']:.1f}%)")
            else:
                print("   ✅ No missing values found!")
        
        # Strategy 1: BMI Imputation (if missing)
        if 'bmi' in missing_analysis:
            if self.show_progress:
                print(f"\n🔧 BMI Imputation Strategy:")
                print("   Method: KNN Imputation using age, gender, and health status")
            
            # Convert to pandas for advanced imputation
            df_pd = df_cleaned.to_pandas()
            
            # Prepare features for BMI imputation
            impute_features = ['age']
            if 'gender' in df_pd.columns:
                # Encode gender for numeric imputation
                le_gender = LabelEncoder()
                df_pd['gender_encoded'] = le_gender.fit_transform(df_pd['gender'].astype(str))
                impute_features.append('gender_encoded')
            
            if 'hypertension' in df_pd.columns:
                impute_features.append('hypertension')
            
            if 'heart_disease' in df_pd.columns:
                impute_features.append('heart_disease')
            
            # KNN Imputation for BMI
            imputer = KNNImputer(n_neighbors=5)
            features_for_imputation = df_pd[impute_features + ['bmi']].copy()
            imputed_features = imputer.fit_transform(features_for_imputation)
            
            # Update BMI column
            df_pd['bmi'] = imputed_features[:, -1]  # BMI is the last column
            
            # Convert back to Polars
            df_cleaned = pl.from_pandas(df_pd.drop('gender_encoded', axis=1, errors='ignore'))
            
            self.missing_strategy_log.append({
                'variable': 'bmi',
                'method': 'KNN Imputation',
                'features_used': impute_features,
                'missing_before': missing_analysis['bmi']['count'],
                'rationale': 'BMI can be predicted from demographic and health factors'
            })
            
            if self.show_progress:
                print(f"   ✅ BMI imputation completed using {len(impute_features)} features")
        
        # Strategy 2: Smoking Status (if missing)
        if 'smoking_status' in missing_analysis:
            if self.show_progress:
                print(f"\n🔧 Smoking Status Strategy:")
                print("   Method: Create 'Unknown' category (clinically meaningful)")
            
            df_cleaned = df_cleaned.with_columns([
                pl.col('smoking_status').fill_null('Unknown')
            ])
            
            self.missing_strategy_log.append({
                'variable': 'smoking_status',
                'method': 'Unknown Category',
                'missing_before': missing_analysis['smoking_status']['count'],
                'rationale': 'Missing smoking data is clinically informative'
            })
            
            if self.show_progress:
                print(f"   ✅ Smoking status: {missing_analysis['smoking_status']['count']} → 'Unknown'")
        
        # Strategy 3: Other variables (if any)
        remaining_missing = {}
        for col in df_cleaned.columns:
            missing_count = df_cleaned[col].null_count()
            if missing_count > 0:
                remaining_missing[col] = missing_count
        
        if remaining_missing:
            if self.show_progress:
                print(f"\n🔧 Handling Remaining Missing Values:")
            
            for col, missing_count in remaining_missing.items():
                if df_cleaned[col].dtype in [pl.Int8, pl.Int16, pl.Int32, pl.Int64, 
                                           pl.Float32, pl.Float64]:
                    # Median imputation for numerical
                    median_val = df_cleaned[col].median()
                    df_cleaned = df_cleaned.with_columns([
                        pl.col(col).fill_null(median_val)
                    ])
                    method = f"Median imputation ({median_val:.2f})"
                else:
                    # Mode imputation for categorical
                    mode_val = df_cleaned[col].mode().first()
                    df_cleaned = df_cleaned.with_columns([
                        pl.col(col).fill_null(mode_val)
                    ])
                    method = f"Mode imputation ('{mode_val}')"
                
                self.missing_strategy_log.append({
                    'variable': col,
                    'method': method,
                    'missing_before': missing_count,
                    'rationale': 'Simple imputation for minimal missing data'
                })
                
                if self.show_progress:
                    print(f"   • {col}: {method}")
        
        # Verify no missing values remain
        final_missing = sum([df_cleaned[col].null_count() for col in df_cleaned.columns])
        
        if self.show_progress:
            print(f"\n✅ Missing Value Strategy Implementation Complete!")
            print(f"   Missing values before: {sum([info['count'] for info in missing_analysis.values()])}")
            print(f"   Missing values after: {final_missing}")
            print(f"   Imputation methods used: {len(self.missing_strategy_log)}")
        
        self.df_cleaned = df_cleaned
        return df_cleaned
    
    def define_research_hypotheses(self) -> Dict:
        """
        Define comprehensive research hypotheses for stroke prediction
        """
        if self.show_progress:
            print("\n" + "="*80)
            print("🔬 RESEARCH HYPOTHESES DEFINITION")
            print("="*80)
        
        hypotheses = {
            'H1_age': {
                'variable': 'age',
                'type': 'continuous',
                'null_hypothesis': 'Age has no association with stroke occurrence',
                'alternative_hypothesis': 'Older age is associated with higher stroke risk',
                'direction': 'one_tailed',
                'test_method': 'Mann-Whitney U (if not normal) or t-test',
                'clinical_importance': 'Age is the strongest known stroke risk factor',
                'expected_effect': 'Large positive association',
                'clinical_threshold': '5+ years difference for clinical significance'
            },
            
            'H2_hypertension': {
                'variable': 'hypertension',
                'type': 'categorical',
                'null_hypothesis': 'Hypertension status is independent of stroke occurrence',
                'alternative_hypothesis': 'Hypertension is associated with increased stroke risk',
                'direction': 'two_tailed',
                'test_method': 'Chi-square test of independence',
                'clinical_importance': 'Major modifiable cardiovascular risk factor',
                'expected_effect': 'Moderate to strong positive association (OR > 2.0)',
                'clinical_threshold': 'OR > 1.5 for clinical relevance'
            },
            
            'H3_heart_disease': {
                'variable': 'heart_disease',
                'type': 'categorical',
                'null_hypothesis': 'Heart disease status is independent of stroke occurrence',
                'alternative_hypothesis': 'Heart disease is associated with increased stroke risk',
                'direction': 'two_tailed',
                'test_method': 'Chi-square test of independence',
                'clinical_importance': 'Cardiovascular comorbidity increases stroke risk',
                'expected_effect': 'Strong positive association (OR > 3.0)',
                'clinical_threshold': 'OR > 2.0 for strong clinical evidence'
            },
            
            'H4_glucose': {
                'variable': 'avg_glucose_level',
                'type': 'continuous',
                'null_hypothesis': 'Average glucose level has no association with stroke',
                'alternative_hypothesis': 'Higher glucose levels are associated with increased stroke risk',
                'direction': 'one_tailed',
                'test_method': 'Mann-Whitney U (if not normal) or t-test',
                'clinical_importance': 'Diabetes/glucose control affects stroke risk',
                'expected_effect': 'Moderate positive association',
                'clinical_threshold': '20+ mg/dL difference for clinical significance'
            },
            
            'H5_bmi': {
                'variable': 'bmi',
                'type': 'continuous',
                'null_hypothesis': 'BMI has no association with stroke occurrence',
                'alternative_hypothesis': 'BMI is associated with stroke risk',
                'direction': 'two_tailed',
                'test_method': 'Mann-Whitney U (if not normal) or t-test',
                'clinical_importance': 'Obesity relationship with stroke is complex (may be U-shaped)',
                'expected_effect': 'Weak to moderate association',
                'clinical_threshold': '2+ kg/m² difference for clinical significance'
            },
            
            'H6_smoking': {
                'variable': 'smoking_status',
                'type': 'categorical',
                'null_hypothesis': 'Smoking status is independent of stroke occurrence',
                'alternative_hypothesis': 'Smoking status is associated with stroke risk',
                'direction': 'two_tailed',
                'test_method': 'Chi-square test of independence',
                'clinical_importance': 'Major modifiable lifestyle risk factor',
                'expected_effect': 'Moderate positive association for current/former smokers',
                'clinical_threshold': 'OR > 1.3 for clinical relevance'
            },
            
            'H7_gender': {
                'variable': 'gender',
                'type': 'categorical',
                'null_hypothesis': 'Gender has no association with stroke occurrence',
                'alternative_hypothesis': 'Gender is associated with different stroke risk',
                'direction': 'two_tailed',
                'test_method': 'Chi-square test of independence',
                'clinical_importance': 'Gender differences in stroke epidemiology',
                'expected_effect': 'Weak to moderate association',
                'clinical_threshold': 'OR > 1.2 for clinical relevance'
            },
            
            'H8_marital_status': {
                'variable': 'ever_married',
                'type': 'categorical',
                'null_hypothesis': 'Marital status is independent of stroke occurrence',
                'alternative_hypothesis': 'Marital status is associated with stroke risk',
                'direction': 'two_tailed',
                'test_method': 'Chi-square test of independence',
                'clinical_importance': 'Social determinants may influence health outcomes',
                'expected_effect': 'Weak association (potential confounding by age)',
                'clinical_threshold': 'OR > 1.2 for clinical relevance'
            },
            
            'H9_work_type': {
                'variable': 'work_type',
                'type': 'categorical',
                'null_hypothesis': 'Work type is independent of stroke occurrence',
                'alternative_hypothesis': 'Work type is associated with stroke risk',
                'direction': 'two_tailed',
                'test_method': 'Chi-square test of independence',
                'clinical_importance': 'Occupational factors and socioeconomic status',
                'expected_effect': 'Weak association',
                'clinical_threshold': 'OR > 1.2 for clinical relevance'
            },
            
            'H10_residence': {
                'variable': 'Residence_type',
                'type': 'categorical',
                'null_hypothesis': 'Residence type is independent of stroke occurrence',
                'alternative_hypothesis': 'Residence type is associated with stroke risk',
                'direction': 'two_tailed',
                'test_method': 'Chi-square test of independence',
                'clinical_importance': 'Urban vs rural healthcare access and lifestyle',
                'expected_effect': 'Weak association',
                'clinical_threshold': 'OR > 1.2 for clinical relevance'
            }
        }
        
        if self.show_progress:
            print(f"\n📋 Research Hypotheses Defined:")
            print(f"   Total hypotheses: {len(hypotheses)}")
            print(f"   Continuous variables: {len([h for h in hypotheses.values() if h['type'] == 'continuous'])}")
            print(f"   Categorical variables: {len([h for h in hypotheses.values() if h['type'] == 'categorical'])}")
            
            print(f"\n🎯 Hypothesis Summary:")
            for h_id, details in list(hypotheses.items())[:5]:  # Show first 5
                print(f"   {h_id}: {details['variable']} - {details['expected_effect']}")
            if len(hypotheses) > 5:
                print(f"   ... and {len(hypotheses) - 5} more")
        
        return hypotheses

In [10]:
# Initialize the testing pipeline
pipeline = StrokeHypothesisTestingPipeline(show_progress=True)
print("🔧 Hypothesis Testing Pipeline initialized!")

🔧 Hypothesis Testing Pipeline initialized!


## 📂 Data Loading and Cleaning Implementation

Loading the dataset and implementing our evidence-based missing value strategy.

In [11]:
# Load the dataset
print("🚀 Starting data loading and cleaning process...")

# Load data (adjust path as needed)
DATASET_PATH = "healthcaredatasetstrokedata 1.csv"
df_original = pipeline.load_data_from_quality_analysis(DATASET_PATH)

# Implement missing value strategy
df_cleaned = pipeline.implement_missing_value_strategy(df_original)

# Define research hypotheses
hypotheses = pipeline.define_research_hypotheses()

print(f"\n🎉 Data preparation completed!")
print(f"   Original shape: {df_original.shape}")
print(f"   Cleaned shape: {df_cleaned.shape}")
print(f"   Research hypotheses: {len(hypotheses)}")

🚀 Starting data loading and cleaning process...
📂 LOADING DATA FROM QUALITY ANALYSIS
✅ Dataset loaded successfully!
   Source: ../data/healthcare-dataset-stroke-data.csv
   Shape: 5,110 rows × 12 columns
   Missing values: 201
   Stroke prevalence: 4.9%

🔧 IMPLEMENTING MISSING VALUE STRATEGY

📊 Missing Values Overview:
   • bmi: 201 (3.9%)

🔧 BMI Imputation Strategy:
   Method: KNN Imputation using age, gender, and health status
   ✅ BMI imputation completed using 4 features

✅ Missing Value Strategy Implementation Complete!
   Missing values before: 201
   Missing values after: 0
   Imputation methods used: 1

🔬 RESEARCH HYPOTHESES DEFINITION

📋 Research Hypotheses Defined:
   Total hypotheses: 10
   Continuous variables: 3
   Categorical variables: 7

🎯 Hypothesis Summary:
   H1_age: age - Large positive association
   H2_hypertension: hypertension - Moderate to strong positive association (OR > 2.0)
   H3_heart_disease: heart_disease - Strong positive association (OR > 3.0)
   H4_gl

## 📊 Statistical Assumptions Testing

Testing normality and other statistical assumptions to guide appropriate test selection.

In [12]:
class NormalityTester:
    """
    Comprehensive normality testing for appropriate statistical test selection
    """
    
    def __init__(self, df: pl.DataFrame, show_progress: bool = True):
        self.df = df
        self.show_progress = show_progress
        self.normality_results = {}
    
    def comprehensive_normality_testing(self) -> Dict:
        """
        Perform comprehensive normality testing for all continuous variables
        """
        if self.show_progress:
            print("="*80)
            print("📊 COMPREHENSIVE NORMALITY TESTING")
            print("="*80)
        
        # Identify continuous variables
        continuous_vars = []
        for col in self.df.columns:
            if self.df[col].dtype in [pl.Int8, pl.Int16, pl.Int32, pl.Int64, 
                                    pl.Float32, pl.Float64]:
                # Check if it's actually continuous (not binary)
                unique_count = self.df[col].n_unique()
                if unique_count > 10:  # More than 10 unique values = continuous
                    continuous_vars.append(col)
        
        if self.show_progress:
            print(f"\nIdentified continuous variables: {continuous_vars}")
        
        for var in continuous_vars:
            if self.show_progress:
                print(f"\n{'='*20} {var.upper()} {'='*20}")
            
            # Extract data
            data = self.df[var].drop_nulls().to_numpy()
            n_samples = len(data)
            
            if n_samples < 3:
                if self.show_progress:
                    print(f"⚠️  Insufficient data for normality testing (n={n_samples})")
                continue
            
            # Initialize results dictionary
            var_results = {
                'variable': var,
                'n_samples': n_samples,
                'mean': np.mean(data),
                'std': np.std(data),
                'skewness': stats.skew(data),
                'kurtosis': stats.kurtosis(data),
                'tests': {}
            }
            
            if self.show_progress:
                print(f"Sample size: {n_samples:,}")
                print(f"Mean: {var_results['mean']:.3f}")
                print(f"Std: {var_results['std']:.3f}")
                print(f"Skewness: {var_results['skewness']:.3f}")
                print(f"Kurtosis: {var_results['kurtosis']:.3f}")
            
            # Test 1: Shapiro-Wilk Test (best for n < 5000)
            if n_samples <= 5000:
                try:
                    shapiro_stat, shapiro_p = shapiro(data)
                    var_results['tests']['shapiro_wilk'] = {
                        'statistic': shapiro_stat,
                        'p_value': shapiro_p,
                        'interpretation': 'Normal' if shapiro_p > 0.05 else 'Non-normal'
                    }
                    
                    if self.show_progress:
                        print(f"Shapiro-Wilk: W = {shapiro_stat:.4f}, p = {shapiro_p:.4f} "
                              f"({'Normal' if shapiro_p > 0.05 else 'Non-normal'})")
                
                except Exception as e:
                    if self.show_progress:
                        print(f"Shapiro-Wilk: Failed ({str(e)})")
            else:
                if self.show_progress:
                    print("Shapiro-Wilk: Skipped (n > 5000)")
            
            # Test 2: Kolmogorov-Smirnov Test
            try:
                ks_stat, ks_p = kstest(data, 'norm', args=(np.mean(data), np.std(data)))
                var_results['tests']['kolmogorov_smirnov'] = {
                    'statistic': ks_stat,
                    'p_value': ks_p,
                    'interpretation': 'Normal' if ks_p > 0.05 else 'Non-normal'
                }
                
                if self.show_progress:
                    print(f"Kolmogorov-Smirnov: D = {ks_stat:.4f}, p = {ks_p:.4f} "
                          f"({'Normal' if ks_p > 0.05 else 'Non-normal'})")
            
            except Exception as e:
                if self.show_progress:
                    print(f"Kolmogorov-Smirnov: Failed ({str(e)})")
            
            # Test 3: Anderson-Darling Test
            try:
                ad_result = anderson(data, dist='norm')
                var_results['tests']['anderson_darling'] = {
                    'statistic': ad_result.statistic,
                    'critical_values': ad_result.critical_values,
                    'significance_levels': ad_result.significance_level,
                    'interpretation': 'Normal' if ad_result.statistic < ad_result.critical_values[2] else 'Non-normal'  # 5% level
                }
                
                if self.show_progress:
                    print(f"Anderson-Darling: A² = {ad_result.statistic:.4f}")
                    print(f"Critical value (5%): {ad_result.critical_values[2]:.4f} "
                          f"({'Normal' if ad_result.statistic < ad_result.critical_values[2] else 'Non-normal'})")
            
            except Exception as e:
                if self.show_progress:
                    print(f"Anderson-Darling: Failed ({str(e)})")
            
            # Test 4: D'Agostino's normality test (for larger samples)
            if n_samples >= 20:
                try:
                    dagostino_stat, dagostino_p = normaltest(data)
                    var_results['tests']['dagostino'] = {
                        'statistic': dagostino_stat,
                        'p_value': dagostino_p,
                        'interpretation': 'Normal' if dagostino_p > 0.05 else 'Non-normal'
                    }
                    
                    if self.show_progress:
                        print(f"D'Agostino: χ² = {dagostino_stat:.4f}, p = {dagostino_p:.4f} "
                              f"({'Normal' if dagostino_p > 0.05 else 'Non-normal'})")
                
                except Exception as e:
                    if self.show_progress:
                        print(f"D'Agostino: Failed ({str(e)})")
            
            # Overall normality assessment
            test_results = []
            for test_name, test_result in var_results['tests'].items():
                if 'interpretation' in test_result:
                    test_results.append(test_result['interpretation'] == 'Normal')
            
            if len(test_results) > 0:
                normal_count = sum(test_results)
                total_tests = len(test_results)
                normality_consensus = normal_count / total_tests
                
                if normality_consensus >= 0.75:
                    overall_assessment = "NORMAL"
                elif normality_consensus >= 0.5:
                    overall_assessment = "QUESTIONABLE"
                else:
                    overall_assessment = "NON-NORMAL"
            else:
                overall_assessment = "INDETERMINATE"
            
            var_results['overall_assessment'] = overall_assessment
            var_results['normality_consensus'] = normality_consensus if 'normality_consensus' in locals() else 0
            
            # Recommended test
            if overall_assessment == "NORMAL":
                recommended_test = "Parametric (t-test, ANOVA)"
            else:
                recommended_test = "Non-parametric (Mann-Whitney U, Kruskal-Wallis)"
            
            var_results['recommended_test'] = recommended_test
            
            if self.show_progress:
                print(f"\n🎯 Overall Assessment: {overall_assessment}")
                print(f"   Normality consensus: {var_results['normality_consensus']:.1%}")
                print(f"   Recommended tests: {recommended_test}")
                
                # Interpretation notes
                if abs(var_results['skewness']) > 1:
                    print(f"   Note: High skewness ({var_results['skewness']:.2f}) suggests non-normality")
                if abs(var_results['kurtosis']) > 1:
                    print(f"   Note: High kurtosis ({var_results['kurtosis']:.2f}) suggests heavy tails")
            
            self.normality_results[var] = var_results
        
        if self.show_progress:
            print(f"\n📋 NORMALITY TESTING SUMMARY:")
            print("-" * 50)
            print(f"{'Variable':<20} {'Assessment':<15} {'Recommended Test':<25}")
            print("-" * 50)
            
            for var, results in self.normality_results.items():
                print(f"{var:<20} {results['overall_assessment']:<15} {results['recommended_test']:<25}")
        
        return self.normality_results

In [13]:
# Perform normality testing
normality_tester = NormalityTester(df_cleaned, show_progress=True)
normality_results = normality_tester.comprehensive_normality_testing()

📊 COMPREHENSIVE NORMALITY TESTING

Identified continuous variables: ['id', 'age', 'avg_glucose_level', 'bmi']

==================== ID ====================
Sample size: 5,110
Mean: 36517.829
Std: 21159.651
Skewness: -0.020
Kurtosis: -1.212
Shapiro-Wilk: Skipped (n > 5000)
Kolmogorov-Smirnov: D = 0.0635, p = 0.0000 (Non-normal)
Anderson-Darling: A² = 59.6234
Critical value (5%): 0.7860 (Non-normal)
D'Agostino: χ² = 4888.4024, p = 0.0000 (Non-normal)

🎯 Overall Assessment: NON-NORMAL
   Normality consensus: 0.0%
   Recommended tests: Non-parametric (Mann-Whitney U, Kruskal-Wallis)
   Note: High kurtosis (-1.21) suggests heavy tails

==================== AGE ====================
Sample size: 5,110
Mean: 43.227
Std: 22.610
Skewness: -0.137
Kurtosis: -0.991
Shapiro-Wilk: Skipped (n > 5000)
Kolmogorov-Smirnov: D = 0.0507, p = 0.0000 (Non-normal)
Anderson-Darling: A² = 33.8564
Critical value (5%): 0.7860 (Non-normal)
D'Agostino: χ² = 1120.5286, p = 0.0000 (Non-normal)

🎯 Overall Assessment: N

## 🔬 Categorical Variables Hypothesis Testing

Comprehensive testing of categorical variables vs stroke outcome using appropriate statistical tests.

In [14]:
class CategoricalHypothesisTester:
    """
    Comprehensive hypothesis testing for categorical variables
    """
    
    def __init__(self, df: pl.DataFrame, normality_results: Dict, show_progress: bool = True):
        self.df = df
        self.normality_results = normality_results
        self.show_progress = show_progress
        self.categorical_results = {}
    
    def test_categorical_associations(self) -> Dict:
        """
        Test associations between categorical variables and stroke outcome
        """
        if self.show_progress:
            print("="*80)
            print("🔬 CATEGORICAL VARIABLES HYPOTHESIS TESTING")
            print("="*80)
        
        # Define categorical variables to test
        categorical_vars = ['gender', 'hypertension', 'heart_disease', 'ever_married', 
                           'work_type', 'Residence_type', 'smoking_status']
        
        # Filter to available variables
        available_vars = [var for var in categorical_vars if var in self.df.columns]
        
        if self.show_progress:
            print(f"Testing {len(available_vars)} categorical variables against stroke outcome")
            print(f"Variables: {available_vars}")
        
        for var in available_vars:
            if self.show_progress:
                print(f"\n{'='*20} {var.upper()} vs STROKE {'='*20}")
            
            self._test_single_categorical_variable(var)
        
        # Summary
        if self.show_progress:
            self._print_categorical_summary()
        
        return self.categorical_results
    
    def _test_single_categorical_variable(self, var: str):
        """
        Comprehensive testing for a single categorical variable
        """
        # Create contingency table
        try:
            contingency_table = (self.df
                               .group_by([var, 'stroke'])
                               .agg(pl.count().alias('count'))
                               .pivot(index=var, columns='stroke', values='count', aggregate_function='first')
                               .fill_null(0)
                               .to_pandas()
                               .set_index(var))
            
            # Ensure we have both stroke classes
            if 0 not in contingency_table.columns:
                contingency_table[0] = 0
            if 1 not in contingency_table.columns:
                contingency_table[1] = 0
            
            contingency_table = contingency_table[[0, 1]]  # Ensure order: no stroke, stroke
            
        except Exception as e:
            if self.show_progress:
                print(f"❌ Error creating contingency table: {e}")
            return
        
        if self.show_progress:
            print("Contingency Table:")
            print(contingency_table)
        
        # Initialize results
        var_results = {
            'variable': var,
            'contingency_table': contingency_table.to_dict(),
            'tests': {}
        }
        
        # Check minimum expected frequencies
        expected_freq = chi2_contingency(contingency_table)[3]
        min_expected = expected_freq.min()
        
        if self.show_progress:
            print(f"\nMinimum expected frequency: {min_expected:.2f}")
        
        # Test 1: Chi-square test of independence
        try:
            chi2_stat, chi2_p, chi2_dof, expected = chi2_contingency(contingency_table)
            
            var_results['tests']['chi_square'] = {
                'statistic': chi2_stat,
                'p_value': chi2_p,
                'degrees_of_freedom': chi2_dof,
                'expected_frequencies': expected.tolist(),
                'min_expected_frequency': min_expected,
                'valid': min_expected >= 5,
                'interpretation': 'Significant' if chi2_p < 0.05 else 'Not significant'
            }
            
            if self.show_progress:
                validity = "✅" if min_expected >= 5 else "⚠️"
                print(f"Chi-square test: {validity}")
                print(f"   χ² = {chi2_stat:.4f}")
                print(f"   df = {chi2_dof}")
                print(f"   p = {chi2_p:.4f}")
                print(f"   Result: {var_results['tests']['chi_square']['interpretation']}")
                
                if min_expected < 5:
                    print(f"   ⚠️  Warning: Low expected frequencies (min = {min_expected:.2f})")
        
        except Exception as e:
            if self.show_progress:
                print(f"❌ Chi-square test failed: {e}")
        
        # Test 2: Fisher's exact test (for 2x2 tables or small frequencies)
        if contingency_table.shape == (2, 2) and min_expected < 5:
            try:
                odds_ratio, fisher_p = fisher_exact(contingency_table)
                
                var_results['tests']['fisher_exact'] = {
                    'odds_ratio': odds_ratio,
                    'p_value': fisher_p,
                    'interpretation': 'Significant' if fisher_p < 0.05 else 'Not significant'
                }
                
                if self.show_progress:
                    print(f"Fisher's exact test: ✅")
                    print(f"   Odds ratio = {odds_ratio:.4f}")
                    print(f"   p = {fisher_p:.4f}")
                    print(f"   Result: {var_results['tests']['fisher_exact']['interpretation']}")
            
            except Exception as e:
                if self.show_progress:
                    print(f"❌ Fisher's exact test failed: {e}")
        
        # Effect size calculations
        self._calculate_categorical_effect_sizes(var, contingency_table, var_results)
        
        # Clinical interpretation
        self._add_clinical_interpretation(var, var_results)
        
        self.categorical_results[var] = var_results
    
    def _calculate_categorical_effect_sizes(self, var: str, contingency_table: pd.DataFrame, var_results: Dict):
        """
        Calculate effect sizes for categorical associations
        """
        try:
            n = contingency_table.sum().sum()
            chi2_stat = var_results['tests']['chi_square']['statistic']
            
            # Cramér's V
            min_dim = min(contingency_table.shape) - 1
            cramers_v = np.sqrt(chi2_stat / (n * min_dim))
            
            var_results['effect_sizes'] = {
                'cramers_v': cramers_v
            }
            
            # Odds ratio (for 2x2 tables)
            if contingency_table.shape == (2, 2):
                a, b = contingency_table.iloc[0, 0], contingency_table.iloc[0, 1]
                c, d = contingency_table.iloc[1, 0], contingency_table.iloc[1, 1]
                
                if b > 0 and c > 0:  # Avoid division by zero
                    odds_ratio = (a * d) / (b * c)
                    
                    # 95% confidence interval for odds ratio
                    log_or = np.log(odds_ratio)
                    se_log_or = np.sqrt(1/a + 1/b + 1/c + 1/d)
                    ci_lower = np.exp(log_or - 1.96 * se_log_or)
                    ci_upper = np.exp(log_or + 1.96 * se_log_or)
                    
                    var_results['effect_sizes']['odds_ratio'] = odds_ratio
                    var_results['effect_sizes']['or_ci_lower'] = ci_lower
                    var_results['effect_sizes']['or_ci_upper'] = ci_upper
                    
                    if self.show_progress:
                        print(f"\nEffect Sizes:")
                        print(f"   Cramér's V = {cramers_v:.4f}")
                        print(f"   Odds Ratio = {odds_ratio:.4f} (95% CI: {ci_lower:.4f}-{ci_upper:.4f})")
                else:
                    if self.show_progress:
                        print(f"\nEffect Sizes:")
                        print(f"   Cramér's V = {cramers_v:.4f}")
                        print(f"   Odds Ratio = Cannot calculate (zero cells)")
            else:
                if self.show_progress:
                    print(f"\nEffect Sizes:")
                    print(f"   Cramér's V = {cramers_v:.4f}")
        
        except Exception as e:
            if self.show_progress:
                print(f"❌ Effect size calculation failed: {e}")
    
    def _add_clinical_interpretation(self, var: str, var_results: Dict):
        """
        Add clinical interpretation of results
        """
        interpretation = {
            'statistical_significance': 'Not significant',
            'clinical_significance': 'No clinical significance',
            'clinical_recommendation': 'No evidence for clinical relevance'
        }
        
        # Check for statistical significance
        if 'chi_square' in var_results['tests']:
            p_value = var_results['tests']['chi_square']['p_value']
            if p_value < 0.05:
                interpretation['statistical_significance'] = 'Statistically significant'
                
                # Assess clinical significance
                if 'odds_ratio' in var_results.get('effect_sizes', {}):
                    or_value = var_results['effect_sizes']['odds_ratio']
                    
                    if or_value > 2.0:
                        interpretation['clinical_significance'] = 'Strong clinical significance'
                        interpretation['clinical_recommendation'] = f'Strong risk factor - OR = {or_value:.2f}'
                    elif or_value > 1.5:
                        interpretation['clinical_significance'] = 'Moderate clinical significance'
                        interpretation['clinical_recommendation'] = f'Moderate risk factor - OR = {or_value:.2f}'
                    elif or_value > 1.2:
                        interpretation['clinical_significance'] = 'Weak clinical significance'
                        interpretation['clinical_recommendation'] = f'Weak risk factor - OR = {or_value:.2f}'
                    elif or_value < 0.8:
                        interpretation['clinical_significance'] = 'Potential protective factor'
                        interpretation['clinical_recommendation'] = f'Potential protective factor - OR = {or_value:.2f}'
                
                # Cramér's V interpretation
                if 'cramers_v' in var_results.get('effect_sizes', {}):
                    cramers_v = var_results['effect_sizes']['cramers_v']
                    if cramers_v > 0.5:
                        interpretation['effect_size_interpretation'] = 'Large effect'
                    elif cramers_v > 0.3:
                        interpretation['effect_size_interpretation'] = 'Medium effect'
                    elif cramers_v > 0.1:
                        interpretation['effect_size_interpretation'] = 'Small effect'
                    else:
                        interpretation['effect_size_interpretation'] = 'Negligible effect'
        
        var_results['clinical_interpretation'] = interpretation
        
        if self.show_progress:
            print(f"\n🏥 Clinical Interpretation:")
            print(f"   Statistical: {interpretation['statistical_significance']}")
            print(f"   Clinical: {interpretation['clinical_significance']}")
            print(f"   Recommendation: {interpretation['clinical_recommendation']}")
    
    def _print_categorical_summary(self):
        """
        Print summary of all categorical tests
        """
        print(f"\n📋 CATEGORICAL HYPOTHESIS TESTING SUMMARY")
        print("="*80)
        
        significant_vars = []
        non_significant_vars = []
        
        for var, results in self.categorical_results.items():
            if 'chi_square' in results['tests']:
                p_value = results['tests']['chi_square']['p_value']
                if p_value < 0.05:
                    significant_vars.append((var, p_value, results.get('effect_sizes', {})))
                else:
                    non_significant_vars.append((var, p_value))
        
        # Sort significant variables by p-value
        significant_vars.sort(key=lambda x: x[1])
        
        print(f"\n✅ SIGNIFICANT ASSOCIATIONS (p < 0.05):")
        if significant_vars:
            print(f"{'Variable':<20} {'p-value':<10} {'Odds Ratio':<12} {'Cramér\'s V':<12} {'Clinical':<15}")
            print("-" * 75)
            
            for var, p_val, effect_sizes in significant_vars:
                or_str = f"{effect_sizes.get('odds_ratio', 'N/A'):.3f}" if isinstance(effect_sizes.get('odds_ratio'), (int, float)) else 'N/A'
                cv_str = f"{effect_sizes.get('cramers_v', 'N/A'):.3f}" if isinstance(effect_sizes.get('cramers_v'), (int, float)) else 'N/A'
                
                # Clinical assessment
                if isinstance(effect_sizes.get('odds_ratio'), (int, float)):
                    or_val = effect_sizes['odds_ratio']
                    if or_val > 2.0:
                        clinical = "Strong"
                    elif or_val > 1.5:
                        clinical = "Moderate"
                    else:
                        clinical = "Weak"
                else:
                    clinical = "Unknown"
                
                print(f"{var:<20} {p_val:<10.4f} {or_str:<12} {cv_str:<12} {clinical:<15}")
        else:
            print("   No significant associations found.")
        
        print(f"\n❌ NON-SIGNIFICANT ASSOCIATIONS (p ≥ 0.05):")
        if non_significant_vars:
            for var, p_val in non_significant_vars:
                print(f"   • {var}: p = {p_val:.4f}")
        else:
            print("   All variables were significant.")


In [15]:
# Perform categorical hypothesis testing
categorical_tester = CategoricalHypothesisTester(df_cleaned, normality_results, show_progress=True)
categorical_results = categorical_tester.test_categorical_associations()

🔬 CATEGORICAL VARIABLES HYPOTHESIS TESTING
Testing 7 categorical variables against stroke outcome
Variables: ['gender', 'hypertension', 'heart_disease', 'ever_married', 'work_type', 'Residence_type', 'smoking_status']

==================== GENDER vs STROKE ====================
Contingency Table:
        0  1
gender      
Male    0  0
Female  0  0
Other   0  0

Minimum expected frequency: nan
Chi-square test: ⚠️
   χ² = nan
   df = 2
   p = nan
   Result: Not significant

Effect Sizes:
   Cramér's V = nan

🏥 Clinical Interpretation:
   Statistical: Not significant
   Clinical: No clinical significance
   Recommendation: No evidence for clinical relevance

==================== HYPERTENSION vs STROKE ====================
Contingency Table:
              0  1
hypertension      
1             0  0
0             0  0

Minimum expected frequency: nan
Chi-square test: ⚠️
   χ² = nan
   df = 1
   p = nan
   Result: Not significant

Effect Sizes:
   Cramér's V = nan
   Odds Ratio = Cannot calcul

## 📊 Continuous Variables Hypothesis Testing

Comprehensive testing of continuous variables vs stroke outcome using appropriate parametric or non-parametric tests.

In [16]:
class ContinuousHypothesisTester:
    """
    Comprehensive hypothesis testing for continuous variables
    """
    
    def __init__(self, df: pl.DataFrame, normality_results: Dict, show_progress: bool = True):
        self.df = df
        self.normality_results = normality_results
        self.show_progress = show_progress
        self.continuous_results = {}
    
    def test_continuous_associations(self) -> Dict:
        """
        Test associations between continuous variables and stroke outcome
        """
        if self.show_progress:
            print("="*80)
            print("📊 CONTINUOUS VARIABLES HYPOTHESIS TESTING")
            print("="*80)
        
        # Get continuous variables from normality testing
        continuous_vars = list(self.normality_results.keys())
        
        if self.show_progress:
            print(f"Testing {len(continuous_vars)} continuous variables against stroke outcome")
            print(f"Variables: {continuous_vars}")
        
        for var in continuous_vars:
            if self.show_progress:
                print(f"\n{'='*20} {var.upper()} vs STROKE {'='*20}")
            
            self._test_single_continuous_variable(var)
        
        # Summary
        if self.show_progress:
            self._print_continuous_summary()
        
        return self.continuous_results
    
    def _test_single_continuous_variable(self, var: str):
        """
        Comprehensive testing for a single continuous variable
        """
        # Extract data by stroke status
        try:
            no_stroke_data = self.df.filter(pl.col('stroke') == 0)[var].drop_nulls().to_numpy()
            stroke_data = self.df.filter(pl.col('stroke') == 1)[var].drop_nulls().to_numpy()
        except Exception as e:
            if self.show_progress:
                print(f"❌ Error extracting data: {e}")
            return
        
        if len(no_stroke_data) == 0 or len(stroke_data) == 0:
            if self.show_progress:
                print(f"❌ Insufficient data for testing")
            return
        
        # Initialize results
        var_results = {
            'variable': var,
            'descriptive_stats': {
                'no_stroke': {
                    'n': len(no_stroke_data),
                    'mean': np.mean(no_stroke_data),
                    'median': np.median(no_stroke_data),
                    'std': np.std(no_stroke_data, ddof=1),
                    'min': np.min(no_stroke_data),
                    'max': np.max(no_stroke_data)
                },
                'stroke': {
                    'n': len(stroke_data),
                    'mean': np.mean(stroke_data),
                    'median': np.median(stroke_data),
                    'std': np.std(stroke_data, ddof=1),
                    'min': np.min(stroke_data),
                    'max': np.max(stroke_data)
                }
            },
            'tests': {}
        }
        
        if self.show_progress:
            print(f"Sample sizes: No stroke = {len(no_stroke_data):,}, Stroke = {len(stroke_data):,}")
            print(f"No stroke:  Mean = {var_results['descriptive_stats']['no_stroke']['mean']:.3f}, "
                  f"SD = {var_results['descriptive_stats']['no_stroke']['std']:.3f}")
            print(f"Stroke:     Mean = {var_results['descriptive_stats']['stroke']['mean']:.3f}, "
                  f"SD = {var_results['descriptive_stats']['stroke']['std']:.3f}")
            
            mean_diff = var_results['descriptive_stats']['stroke']['mean'] - var_results['descriptive_stats']['no_stroke']['mean']
            print(f"Mean difference: {mean_diff:.3f}")
        
        # Determine test strategy based on normality
        normality_info = self.normality_results.get(var, {})
        use_parametric = normality_info.get('overall_assessment') == 'NORMAL'
        
        if self.show_progress:
            test_strategy = "Parametric" if use_parametric else "Non-parametric"
            print(f"\nTest strategy: {test_strategy} (based on normality assessment)")
        
        # Parametric tests
        if use_parametric:
            self._perform_parametric_tests(var, no_stroke_data, stroke_data, var_results)
        
        # Non-parametric tests (always perform for comparison)
        self._perform_nonparametric_tests(var, no_stroke_data, stroke_data, var_results)
        
        # Effect sizes
        self._calculate_continuous_effect_sizes(var, no_stroke_data, stroke_data, var_results)
        
        # Clinical interpretation
        self._add_continuous_clinical_interpretation(var, var_results)
        
        self.continuous_results[var] = var_results
    
    def _perform_parametric_tests(self, var: str, no_stroke_data: np.ndarray, stroke_data: np.ndarray, var_results: Dict):
        """
        Perform parametric tests (t-tests)
        """
        if self.show_progress:
            print(f"\n📊 Parametric Tests:")
        
        # Test for equal variances (Levene's test)
        try:
            levene_stat, levene_p = levene(no_stroke_data, stroke_data)
            equal_variances = levene_p > 0.05
            
            var_results['tests']['levene_test'] = {
                'statistic': levene_stat,
                'p_value': levene_p,
                'equal_variances': equal_variances,
                'interpretation': 'Equal variances' if equal_variances else 'Unequal variances'
            }
            
            if self.show_progress:
                print(f"   Levene's test: F = {levene_stat:.4f}, p = {levene_p:.4f} "
                      f"({'Equal variances' if equal_variances else 'Unequal variances'})")
        
        except Exception as e:
            if self.show_progress:
                print(f"   ❌ Levene's test failed: {e}")
            equal_variances = True  # Default assumption
        
        # Independent samples t-test
        try:
            t_stat, t_p = ttest_ind(stroke_data, no_stroke_data, equal_var=equal_variances)
            
            test_type = "Student's t-test" if equal_variances else "Welch's t-test"
            
            var_results['tests']['t_test'] = {
                'test_type': test_type,
                'statistic': t_stat,
                'p_value': t_p,
                'equal_var_assumed': equal_variances,
                'interpretation': 'Significant' if t_p < 0.05 else 'Not significant'
            }
            
            if self.show_progress:
                print(f"   {test_type}: t = {t_stat:.4f}, p = {t_p:.4f}")
                print(f"   Result: {var_results['tests']['t_test']['interpretation']}")
        
        except Exception as e:
            if self.show_progress:
                print(f"   ❌ t-test failed: {e}")
    
    def _perform_nonparametric_tests(self, var: str, no_stroke_data: np.ndarray, stroke_data: np.ndarray, var_results: Dict):
        """
        Perform non-parametric tests (Mann-Whitney U)
        """
        if self.show_progress:
            print(f"\n📊 Non-parametric Tests:")
        
        # Mann-Whitney U test
        try:
            u_stat, u_p = mannwhitneyu(stroke_data, no_stroke_data, alternative='two-sided')
            
            var_results['tests']['mann_whitney'] = {
                'statistic': u_stat,
                'p_value': u_p,
                'interpretation': 'Significant' if u_p < 0.05 else 'Not significant'
            }
            
            if self.show_progress:
                print(f"   Mann-Whitney U: U = {u_stat:.1f}, p = {u_p:.4f}")
                print(f"   Result: {var_results['tests']['mann_whitney']['interpretation']}")
        
        except Exception as e:
            if self.show_progress:
                print(f"   ❌ Mann-Whitney U test failed: {e}")
    
    def _calculate_continuous_effect_sizes(self, var: str, no_stroke_data: np.ndarray, stroke_data: np.ndarray, var_results: Dict):
        """
        Calculate effect sizes for continuous variables
        """
        try:
            # Cohen's d
            mean_diff = np.mean(stroke_data) - np.mean(no_stroke_data)
            pooled_std = np.sqrt(((len(no_stroke_data) - 1) * np.var(no_stroke_data, ddof=1) + 
                                (len(stroke_data) - 1) * np.var(stroke_data, ddof=1)) / 
                               (len(no_stroke_data) + len(stroke_data) - 2))
            
            cohens_d = mean_diff / pooled_std if pooled_std > 0 else 0
            
            # Glass's delta (using control group SD)
            glass_delta = mean_diff / np.std(no_stroke_data, ddof=1) if np.std(no_stroke_data, ddof=1) > 0 else 0
            
            # Point-biserial correlation
            all_data = np.concatenate([no_stroke_data, stroke_data])
            group_labels = np.concatenate([np.zeros(len(no_stroke_data)), np.ones(len(stroke_data))])
            
            point_biserial_r, pb_p = pointbiserialr(group_labels, all_data)
            
            # Effect size for Mann-Whitney (r = Z / sqrt(N))
            if 'mann_whitney' in var_results['tests']:
                n1, n2 = len(stroke_data), len(no_stroke_data)
                # Approximate z-score from U statistic
                u_stat = var_results['tests']['mann_whitney']['statistic']
                expected_u = n1 * n2 / 2
                std_u = np.sqrt(n1 * n2 * (n1 + n2 + 1) / 12)
                z_score = (u_stat - expected_u) / std_u if std_u > 0 else 0
                effect_size_r = abs(z_score) / np.sqrt(n1 + n2)
            else:
                effect_size_r = 0
            
            var_results['effect_sizes'] = {
                'cohens_d': cohens_d,
                'glass_delta': glass_delta,
                'point_biserial_r': point_biserial_r,
                'point_biserial_p': pb_p,
                'mann_whitney_r': effect_size_r,
                'mean_difference': mean_diff,
                'pooled_std': pooled_std
            }
            
            if self.show_progress:
                print(f"\n📏 Effect Sizes:")
                print(f"   Cohen's d = {cohens_d:.4f}")
                print(f"   Point-biserial r = {point_biserial_r:.4f} (p = {pb_p:.4f})")
                print(f"   Mean difference = {mean_diff:.3f}")
                print(f"   Mann-Whitney r = {effect_size_r:.4f}")
        
        except Exception as e:
            if self.show_progress:
                print(f"❌ Effect size calculation failed: {e}")
    
    def _add_continuous_clinical_interpretation(self, var: str, var_results: Dict):
        """
        Add clinical interpretation for continuous variables
        """
        interpretation = {
            'statistical_significance': 'Not significant',
            'clinical_significance': 'No clinical significance',
            'effect_size_interpretation': 'Negligible effect',
            'clinical_recommendation': 'No evidence for clinical relevance'
        }
        
        # Check statistical significance (prioritize non-parametric for robustness)
        p_value = None
        if 'mann_whitney' in var_results['tests']:
            p_value = var_results['tests']['mann_whitney']['p_value']
        elif 't_test' in var_results['tests']:
            p_value = var_results['tests']['t_test']['p_value']
        
        if p_value is not None and p_value < 0.05:
            interpretation['statistical_significance'] = 'Statistically significant'
            
            # Effect size interpretation
            if 'cohens_d' in var_results.get('effect_sizes', {}):
                cohens_d = abs(var_results['effect_sizes']['cohens_d'])
                
                if cohens_d >= 0.8:
                    interpretation['effect_size_interpretation'] = 'Large effect'
                elif cohens_d >= 0.5:
                    interpretation['effect_size_interpretation'] = 'Medium effect'
                elif cohens_d >= 0.2:
                    interpretation['effect_size_interpretation'] = 'Small effect'
                else:
                    interpretation['effect_size_interpretation'] = 'Negligible effect'
            
            # Clinical significance based on variable-specific thresholds
            mean_diff = var_results.get('effect_sizes', {}).get('mean_difference', 0)
            
            if var.lower() == 'age':
                if abs(mean_diff) >= 10:
                    interpretation['clinical_significance'] = 'Strong clinical significance'
                    interpretation['clinical_recommendation'] = f'Major age difference: {mean_diff:.1f} years'
                elif abs(mean_diff) >= 5:
                    interpretation['clinical_significance'] = 'Moderate clinical significance'
                    interpretation['clinical_recommendation'] = f'Meaningful age difference: {mean_diff:.1f} years'
                else:
                    interpretation['clinical_significance'] = 'Weak clinical significance'
                    interpretation['clinical_recommendation'] = f'Small age difference: {mean_diff:.1f} years'
            
            elif 'glucose' in var.lower():
                if abs(mean_diff) >= 30:
                    interpretation['clinical_significance'] = 'Strong clinical significance'
                    interpretation['clinical_recommendation'] = f'Major glucose difference: {mean_diff:.1f} mg/dL'
                elif abs(mean_diff) >= 20:
                    interpretation['clinical_significance'] = 'Moderate clinical significance'
                    interpretation['clinical_recommendation'] = f'Meaningful glucose difference: {mean_diff:.1f} mg/dL'
                else:
                    interpretation['clinical_significance'] = 'Weak clinical significance'
                    interpretation['clinical_recommendation'] = f'Small glucose difference: {mean_diff:.1f} mg/dL'
            
            elif 'bmi' in var.lower():
                if abs(mean_diff) >= 3:
                    interpretation['clinical_significance'] = 'Moderate clinical significance'
                    interpretation['clinical_recommendation'] = f'Meaningful BMI difference: {mean_diff:.1f} kg/m²'
                elif abs(mean_diff) >= 2:
                    interpretation['clinical_significance'] = 'Weak clinical significance'
                    interpretation['clinical_recommendation'] = f'Small BMI difference: {mean_diff:.1f} kg/m²'
                else:
                    interpretation['clinical_significance'] = 'No clinical significance'
                    interpretation['clinical_recommendation'] = f'Trivial BMI difference: {mean_diff:.1f} kg/m²'
            
            else:
                # Generic interpretation
                if abs(mean_diff) > 0:
                    interpretation['clinical_recommendation'] = f'Mean difference: {mean_diff:.3f}'
        
        var_results['clinical_interpretation'] = interpretation
        
        if self.show_progress:
            print(f"\n🏥 Clinical Interpretation:")
            print(f"   Statistical: {interpretation['statistical_significance']}")
            print(f"   Effect size: {interpretation['effect_size_interpretation']}")
            print(f"   Clinical: {interpretation['clinical_significance']}")
            print(f"   Recommendation: {interpretation['clinical_recommendation']}")
    
    def _print_continuous_summary(self):
        """
        Print summary of all continuous tests
        """
        print(f"\n📋 CONTINUOUS HYPOTHESIS TESTING SUMMARY")
        print("="*80)
        
        significant_vars = []
        non_significant_vars = []
        
        for var, results in self.continuous_results.items():
            # Use Mann-Whitney p-value as primary (more robust)
            p_value = None
            if 'mann_whitney' in results['tests']:
                p_value = results['tests']['mann_whitney']['p_value']
            elif 't_test' in results['tests']:
                p_value = results['tests']['t_test']['p_value']
            
            if p_value is not None:
                if p_value < 0.05:
                    effect_sizes = results.get('effect_sizes', {})
                    significant_vars.append((var, p_value, effect_sizes))
                else:
                    non_significant_vars.append((var, p_value))
        
        # Sort significant variables by p-value
        significant_vars.sort(key=lambda x: x[1])
        
        print(f"\n✅ SIGNIFICANT ASSOCIATIONS (p < 0.05):")
        if significant_vars:
            print(f"{'Variable':<20} {'p-value':<10} {'Cohen\'s d':<12} {'Mean Diff':<12} {'Clinical':<15}")
            print("-" * 75)
            
            for var, p_val, effect_sizes in significant_vars:
                cohens_d = effect_sizes.get('cohens_d', 0)
                mean_diff = effect_sizes.get('mean_difference', 0)
                
                # Clinical assessment
                if var.lower() == 'age' and abs(mean_diff) >= 5:
                    clinical = "Strong"
                elif 'glucose' in var.lower() and abs(mean_diff) >= 20:
                    clinical = "Strong"
                elif 'bmi' in var.lower() and abs(mean_diff) >= 2:
                    clinical = "Moderate"
                elif abs(cohens_d) >= 0.5:
                    clinical = "Moderate"
                elif abs(cohens_d) >= 0.2:
                    clinical = "Weak"
                else:
                    clinical = "Minimal"
                
                print(f"{var:<20} {p_val:<10.4f} {cohens_d:<12.3f} {mean_diff:<12.3f} {clinical:<15}")
        else:
            print("   No significant associations found.")
        
        print(f"\n❌ NON-SIGNIFICANT ASSOCIATIONS (p ≥ 0.05):")
        if non_significant_vars:
            for var, p_val in non_significant_vars:
                print(f"   • {var}: p = {p_val:.4f}")
        else:
            print("   All variables were significant.")

In [17]:
# Perform continuous hypothesis testing
continuous_tester = ContinuousHypothesisTester(df_cleaned, normality_results, show_progress=True)
continuous_results = continuous_tester.test_continuous_associations()

📊 CONTINUOUS VARIABLES HYPOTHESIS TESTING
Testing 4 continuous variables against stroke outcome
Variables: ['id', 'age', 'avg_glucose_level', 'bmi']

==================== ID vs STROKE ====================
Sample sizes: No stroke = 4,861, Stroke = 249
No stroke:  Mean = 36487.236, SD = 21120.133
Stroke:     Mean = 37115.068, SD = 21993.345
Mean difference: 627.832

Test strategy: Non-parametric (based on normality assessment)

📊 Non-parametric Tests:
   Mann-Whitney U: U = 615742.0, p = 0.6423
   Result: Not significant

📏 Effect Sizes:
   Cohen's d = 0.0297
   Point-biserial r = 0.0064 (p = 0.6480)
   Mean difference = 627.832
   Mann-Whitney r = 0.0065

🏥 Clinical Interpretation:
   Statistical: Not significant
   Effect size: Negligible effect
   Clinical: No clinical significance
   Recommendation: No evidence for clinical relevance

==================== AGE vs STROKE ====================
Sample sizes: No stroke = 4,861, Stroke = 249
No stroke:  Mean = 41.972, SD = 22.292
Stroke:   

## 🔬 Multiple Testing Correction

Applying appropriate corrections for multiple hypothesis testing to control family-wise error rate.

In [18]:
class MultipleTestingCorrector:
    """
    Apply multiple testing corrections to control Type I error rate
    """
    
    def __init__(self, categorical_results: Dict, continuous_results: Dict, show_progress: bool = True):
        self.categorical_results = categorical_results
        self.continuous_results = continuous_results
        self.show_progress = show_progress
        self.correction_results = {}
    
    def apply_multiple_testing_correction(self, alpha: float = 0.05) -> Dict:
        """
        Apply multiple testing corrections
        """
        if self.show_progress:
            print("="*80)
            print("🔬 MULTIPLE TESTING CORRECTION")
            print("="*80)
        
        # Collect all p-values and test information
        all_tests = []
        
        # Categorical tests
        for var, results in self.categorical_results.items():
            if 'chi_square' in results['tests']:
                p_value = results['tests']['chi_square']['p_value']
                all_tests.append({
                    'variable': var,
                    'test_type': 'Chi-square',
                    'original_p': p_value,
                    'category': 'categorical'
                })
        
        # Continuous tests
        for var, results in self.continuous_results.items():
            # Use Mann-Whitney as primary test (more robust)
            if 'mann_whitney' in results['tests']:
                p_value = results['tests']['mann_whitney']['p_value']
                test_type = 'Mann-Whitney U'
            elif 't_test' in results['tests']:
                p_value = results['tests']['t_test']['p_value']
                test_type = 't-test'
            else:
                continue
            
            all_tests.append({
                'variable': var,
                'test_type': test_type,
                'original_p': p_value,
                'category': 'continuous'
            })
        
        if not all_tests:
            if self.show_progress:
                print("❌ No test results available for correction")
            return {}
        
        # Extract p-values
        p_values = [test['original_p'] for test in all_tests]
        
        if self.show_progress:
            print(f"Total tests performed: {len(all_tests)}")
            print(f"Original significance level (α): {alpha}")
        
        # Apply different correction methods
        corrections = {}
        
        # 1. Bonferroni correction
        bonferroni_alpha = alpha / len(p_values)
        bonferroni_significant = [p < bonferroni_alpha for p in p_values]
        
        corrections['bonferroni'] = {
            'corrected_alpha': bonferroni_alpha,
            'significant': bonferroni_significant,
            'method_description': 'Conservative family-wise error rate control'
        }
        
        # 2. Holm-Bonferroni correction
        try:
            holm_rejected, holm_p_corrected, _, holm_alpha = multipletests(p_values, alpha=alpha, method='holm')
            corrections['holm'] = {
                'rejected': holm_rejected,
                'corrected_p': holm_p_corrected,
                'method_description': 'Step-down Bonferroni method'
            }
        except Exception as e:
            if self.show_progress:
                print(f"⚠️  Holm correction failed: {e}")
        
        # 3. Benjamini-Hochberg (FDR) correction
        try:
            fdr_rejected, fdr_p_corrected, _, fdr_alpha = multipletests(p_values, alpha=alpha, method='fdr_bh')
            corrections['benjamini_hochberg'] = {
                'rejected': fdr_rejected,
                'corrected_p': fdr_p_corrected,
                'method_description': 'False Discovery Rate control (less conservative)'
            }
        except Exception as e:
            if self.show_progress:
                print(f"⚠️  FDR correction failed: {e}")
        
        # 4. Sidak correction
        try:
            sidak_rejected, sidak_p_corrected, _, sidak_alpha = multipletests(p_values, alpha=alpha, method='sidak')
            corrections['sidak'] = {
                'rejected': sidak_rejected,
                'corrected_p': sidak_p_corrected,
                'method_description': 'Less conservative than Bonferroni'
            }
        except Exception as e:
            if self.show_progress:
                print(f"⚠️  Sidak correction failed: {e}")
        
        # Combine results
        for i, test in enumerate(all_tests):
            test_result = {
                'variable': test['variable'],
                'test_type': test['test_type'],
                'category': test['category'],
                'original_p': test['original_p'],
                'original_significant': test['original_p'] < alpha
            }
            
            # Add correction results
            for method, correction in corrections.items():
                if method == 'bonferroni':
                    test_result[f'{method}_significant'] = correction['significant'][i]
                    test_result[f'{method}_alpha'] = correction['corrected_alpha']
                else:
                    if 'rejected' in correction:
                        test_result[f'{method}_significant'] = correction['rejected'][i]
                        test_result[f'{method}_corrected_p'] = correction['corrected_p'][i]
            
            self.correction_results[test['variable']] = test_result
        
        # Print results
        if self.show_progress:
            self._print_correction_summary(corrections, alpha)
        
        return self.correction_results
    
    def _print_correction_summary(self, corrections: Dict, alpha: float):
        """
        Print comprehensive summary of multiple testing corrections
        """
        print(f"\n📊 CORRECTION METHODS APPLIED:")
        print("-" * 50)
        
        for method, correction in corrections.items():
            print(f"\n{method.upper()}:")
            print(f"   Description: {correction['method_description']}")
            
            if method == 'bonferroni':
                print(f"   Corrected α: {correction['corrected_alpha']:.6f}")
                sig_count = sum(correction['significant'])
            else:
                if 'rejected' in correction:
                    sig_count = sum(correction['rejected'])
                else:
                    sig_count = 0
            
            print(f"   Significant tests: {sig_count}/{len(self.correction_results)}")
        
        print(f"\n📋 DETAILED RESULTS:")
        print("-" * 80)
        print(f"{'Variable':<20} {'Test':<15} {'Original p':<12} {'Original':<10} {'Bonferroni':<12} {'FDR':<8}")
        print("-" * 80)
        
        # Sort by original p-value
        sorted_results = sorted(self.correction_results.values(), key=lambda x: x['original_p'])
        
        for result in sorted_results:
            var = result['variable']
            test_type = result['test_type']
            orig_p = result['original_p']
            orig_sig = "✅" if result['original_significant'] else "❌"
            
            bonf_sig = "✅" if result.get('bonferroni_significant', False) else "❌"
            fdr_sig = "✅" if result.get('benjamini_hochberg_significant', False) else "❌"
            
            print(f"{var:<20} {test_type:<15} {orig_p:<12.4f} {orig_sig:<10} {bonf_sig:<12} {fdr_sig:<8}")
        
        # Summary statistics
        original_sig = sum([r['original_significant'] for r in self.correction_results.values()])
        bonf_sig = sum([r.get('bonferroni_significant', False) for r in self.correction_results.values()])
        fdr_sig = sum([r.get('benjamini_hochberg_significant', False) for r in self.correction_results.values()])
        
        print(f"\n📈 SUMMARY:")
        print(f"   Original significance: {original_sig}/{len(self.correction_results)} tests")
        print(f"   After Bonferroni: {bonf_sig}/{len(self.correction_results)} tests")
        print(f"   After FDR: {fdr_sig}/{len(self.correction_results)} tests")
        
        # Recommendations
        print(f"\n💡 RECOMMENDATIONS:")
        if bonf_sig > 0:
            print(f"   ✅ {bonf_sig} variables survive conservative Bonferroni correction")
            print(f"   → These represent the strongest evidence for association")
        
        if fdr_sig > bonf_sig:
            print(f"   ⚠️  {fdr_sig - bonf_sig} additional variables significant with FDR")
            print(f"   → Consider these for exploratory analysis")
        
        if original_sig > fdr_sig:
            print(f"   ⚠️  {original_sig - fdr_sig} variables significant only without correction")
            print(f"   → May represent false discoveries due to multiple testing")

In [19]:
# Apply multiple testing correction
corrector = MultipleTestingCorrector(categorical_results, continuous_results, show_progress=True)
correction_results = corrector.apply_multiple_testing_correction(alpha=0.05)

🔬 MULTIPLE TESTING CORRECTION
Total tests performed: 11
Original significance level (α): 0.05

📊 CORRECTION METHODS APPLIED:
--------------------------------------------------

BONFERRONI:
   Description: Conservative family-wise error rate control
   Corrected α: 0.004545
   Significant tests: 3/11

HOLM:
   Description: Step-down Bonferroni method
   Significant tests: 3/11

BENJAMINI_HOCHBERG:
   Description: False Discovery Rate control (less conservative)
   Significant tests: 3/11

SIDAK:
   Description: Less conservative than Bonferroni
   Significant tests: 3/11

📋 DETAILED RESULTS:
--------------------------------------------------------------------------------
Variable             Test            Original p   Original   Bonferroni   FDR     
--------------------------------------------------------------------------------
gender               Chi-square      nan          ❌          ❌            ❌       
hypertension         Chi-square      nan          ❌          ❌            

## 📊 Comprehensive Results Visualization

Creating comprehensive visualizations to illustrate hypothesis testing findings and statistical evidence.

In [20]:
def create_hypothesis_testing_visualizations(categorical_results, continuous_results, correction_results):
    """
    Create comprehensive visualizations for hypothesis testing results
    """
    print("="*80)
    print("📊 CREATING HYPOTHESIS TESTING VISUALIZATIONS")
    print("="*80)
    
    visualizations = {}
    
    # 1. P-values visualization (Volcano plot style)
    print("Creating p-values visualization...")
    
    # Collect data for visualization
    plot_data = []
    
    # Categorical variables
    for var, results in categorical_results.items():
        if 'chi_square' in results['tests']:
            p_val = results['tests']['chi_square']['p_value']
            effect_size = results.get('effect_sizes', {}).get('cramers_v', 0)
            
            plot_data.append({
                'variable': var,
                'p_value': p_val,
                'log_p': -np.log10(p_val),
                'effect_size': effect_size,
                'type': 'Categorical',
                'significant': p_val < 0.05,
                'bonferroni_sig': correction_results.get(var, {}).get('bonferroni_significant', False)
            })
    
    # Continuous variables
    for var, results in continuous_results.items():
        p_val = None
        if 'mann_whitney' in results['tests']:
            p_val = results['tests']['mann_whitney']['p_value']
        elif 't_test' in results['tests']:
            p_val = results['tests']['t_test']['p_value']
        
        if p_val is not None:
            effect_size = abs(results.get('effect_sizes', {}).get('cohens_d', 0))
            
            plot_data.append({
                'variable': var,
                'p_value': p_val,
                'log_p': -np.log10(p_val),
                'effect_size': effect_size,
                'type': 'Continuous',
                'significant': p_val < 0.05,
                'bonferroni_sig': correction_results.get(var, {}).get('bonferroni_significant', False)
            })
    
    if plot_data:
        plot_df = pd.DataFrame(plot_data)
        
        # Create volcano plot
        fig_volcano = px.scatter(
            plot_df,
            x='effect_size',
            y='log_p',
            color='bonferroni_sig',
            symbol='type',
            hover_data=['variable', 'p_value'],
            title='Hypothesis Testing Results: Effect Size vs Statistical Significance',
            labels={
                'effect_size': 'Effect Size (Cramér\'s V or |Cohen\'s d|)',
                'log_p': '-log₁₀(p-value)',
                'bonferroni_sig': 'Bonferroni Significant'
            },
            color_discrete_map={True: 'red', False: 'lightblue'}
        )
        
        # Add significance lines
        fig_volcano.add_hline(y=-np.log10(0.05), line_dash="dash", line_color="red", 
                             annotation_text="p = 0.05")
        fig_volcano.add_hline(y=-np.log10(0.05/len(plot_data)), line_dash="dash", line_color="orange",
                             annotation_text="Bonferroni corrected")
        fig_volcano.add_vline(x=0.3, line_dash="dash", line_color="green",
                             annotation_text="Medium effect")
        
        fig_volcano.update_layout(height=600, width=800)
        visualizations['volcano_plot'] = fig_volcano
        fig_volcano.show()
    
    # 2. Effect sizes comparison
    print("Creating effect sizes comparison...")
    
    effect_data = []
    for var, results in categorical_results.items():
        if 'effect_sizes' in results:
            if 'odds_ratio' in results['effect_sizes']:
                or_val = results['effect_sizes']['odds_ratio']
                effect_data.append({
                    'variable': var,
                    'effect_measure': 'Odds Ratio',
                    'effect_value': or_val,
                    'log_effect': np.log(or_val) if or_val > 0 else 0,
                    'type': 'Categorical',
                    'significant': results['tests']['chi_square']['p_value'] < 0.05
                })
    
    for var, results in continuous_results.items():
        if 'effect_sizes' in results:
            cohens_d = results['effect_sizes'].get('cohens_d', 0)
            effect_data.append({
                'variable': var,
                'effect_measure': 'Cohen\'s d',
                'effect_value': abs(cohens_d),
                'log_effect': cohens_d,  # Keep sign for Cohen's d
                'type': 'Continuous',
                'significant': results['tests'].get('mann_whitney', {}).get('p_value', 1) < 0.05 or 
                              results['tests'].get('t_test', {}).get('p_value', 1) < 0.05
            })
    
    if effect_data:
        effect_df = pd.DataFrame(effect_data)
        
        # Separate plots for different effect measures
        fig_effects = make_subplots(
            rows=1, cols=2,
            subplot_titles=['Odds Ratios (Categorical)', 'Cohen\'s d (Continuous)'],
            specs=[[{"secondary_y": False}, {"secondary_y": False}]]
        )
        
        # Odds ratios
        or_data = effect_df[effect_df['effect_measure'] == 'Odds Ratio']
        if len(or_data) > 0:
            colors = ['red' if sig else 'lightblue' for sig in or_data['significant']]
            fig_effects.add_trace(
                go.Bar(
                    x=or_data['variable'],
                    y=or_data['effect_value'],
                    name='Odds Ratio',
                    marker_color=colors,
                    showlegend=False
                ),
                row=1, col=1
            )
            fig_effects.add_hline(y=1, line_dash="dash", line_color="black", row=1, col=1)
        
        # Cohen's d
        cohens_data = effect_df[effect_df['effect_measure'] == 'Cohen\'s d']
        if len(cohens_data) > 0:
            colors = ['red' if sig else 'lightblue' for sig in cohens_data['significant']]
            fig_effects.add_trace(
                go.Bar(
                    x=cohens_data['variable'],
                    y=cohens_data['log_effect'],
                    name='Cohen\'s d',
                    marker_color=colors,
                    showlegend=False
                ),
                row=1, col=2
            )
            fig_effects.add_hline(y=0, line_dash="dash", line_color="black", row=1, col=2)
            fig_effects.add_hline(y=0.2, line_dash="dot", line_color="green", row=1, col=2)
            fig_effects.add_hline(y=0.5, line_dash="dot", line_color="orange", row=1, col=2)
            fig_effects.add_hline(y=0.8, line_dash="dot", line_color="red", row=1, col=2)
        
        fig_effects.update_layout(
            title='Effect Sizes by Variable Type',
            height=500,
            showlegend=False
        )
        fig_effects.update_xaxes(tickangle=45)
        
        visualizations['effect_sizes'] = fig_effects
        fig_effects.show()
    
    # 3. Multiple testing correction comparison
    print("Creating multiple testing correction visualization...")
    
    if correction_results:
        correction_data = []
        for var, results in correction_results.items():
            correction_data.append({
                'variable': var,
                'original_p': results['original_p'],
                'original_sig': results['original_significant'],
                'bonferroni_sig': results.get('bonferroni_significant', False),
                'fdr_sig': results.get('benjamini_hochberg_significant', False)
            })
        
        correction_df = pd.DataFrame(correction_data)
        correction_df = correction_df.sort_values('original_p')
        
        # Create correction comparison plot
        fig_correction = go.Figure()
        
        # Add bars for each correction method
        x_pos = list(range(len(correction_df)))
        
        fig_correction.add_trace(go.Bar(
            x=correction_df['variable'],
            y=[1 if sig else 0 for sig in correction_df['original_sig']],
            name='Original (α = 0.05)',
            marker_color='lightblue',
            opacity=0.7
        ))
        
        fig_correction.add_trace(go.Bar(
            x=correction_df['variable'],
            y=[1 if sig else 0 for sig in correction_df['bonferroni_sig']],
            name='Bonferroni',
            marker_color='red',
            opacity=0.7
        ))
        
        fig_correction.add_trace(go.Bar(
            x=correction_df['variable'],
            y=[1 if sig else 0 for sig in correction_df['fdr_sig']],
            name='FDR (Benjamini-Hochberg)',
            marker_color='orange',
            opacity=0.7
        ))
        
        fig_correction.update_layout(
            title='Multiple Testing Correction Comparison',
            xaxis_title='Variables',
            yaxis_title='Significant (1) / Not Significant (0)',
            barmode='group',
            height=500,
            xaxis_tickangle=45
        )
        
        visualizations['correction_comparison'] = fig_correction
        fig_correction.show()
    
    # 4. Clinical significance assessment
    print("Creating clinical significance assessment...")
    
    clinical_data = []
    
    # Collect clinical significance data
    for var, results in categorical_results.items():
        if 'clinical_interpretation' in results:
            clinical_sig = results['clinical_interpretation']['clinical_significance']
            stat_sig = results['clinical_interpretation']['statistical_significance']
            
            clinical_score = 0
            if 'Strong' in clinical_sig:
                clinical_score = 3
            elif 'Moderate' in clinical_sig:
                clinical_score = 2
            elif 'Weak' in clinical_sig:
                clinical_score = 1
            
            clinical_data.append({
                'variable': var,
                'type': 'Categorical',
                'statistical_significance': 1 if 'Significant' in stat_sig else 0,
                'clinical_significance': clinical_score,
                'category': clinical_sig
            })
    
    for var, results in continuous_results.items():
        if 'clinical_interpretation' in results:
            clinical_sig = results['clinical_interpretation']['clinical_significance']
            stat_sig = results['clinical_interpretation']['statistical_significance']
            
            clinical_score = 0
            if 'Strong' in clinical_sig:
                clinical_score = 3
            elif 'Moderate' in clinical_sig:
                clinical_score = 2
            elif 'Weak' in clinical_sig:
                clinical_score = 1
            
            clinical_data.append({
                'variable': var,
                'type': 'Continuous',
                'statistical_significance': 1 if 'Significant' in stat_sig else 0,
                'clinical_significance': clinical_score,
                'category': clinical_sig
            })
    
    if clinical_data:
        clinical_df = pd.DataFrame(clinical_data)
        
        # Create clinical vs statistical significance scatter plot
        fig_clinical = px.scatter(
            clinical_df,
            x='statistical_significance',
            y='clinical_significance',
            color='type',
            symbol='type',
            hover_data=['variable', 'category'],
            title='Statistical vs Clinical Significance',
            labels={
                'statistical_significance': 'Statistical Significance',
                'clinical_significance': 'Clinical Significance Score'
            }
        )
        
        # Add quadrant lines
        fig_clinical.add_vline(x=0.5, line_dash="dash", line_color="gray")
        fig_clinical.add_hline(y=1.5, line_dash="dash", line_color="gray")
        
        # Add quadrant labels
        fig_clinical.add_annotation(x=0.25, y=2.5, text="Clinically<br>Important<br>Only", showarrow=False)
        fig_clinical.add_annotation(x=0.75, y=2.5, text="Both<br>Statistically &<br>Clinically<br>Significant", showarrow=False)
        fig_clinical.add_annotation(x=0.25, y=0.5, text="Neither<br>Significant", showarrow=False)
        fig_clinical.add_annotation(x=0.75, y=0.5, text="Statistically<br>Significant<br>Only", showarrow=False)
        
        fig_clinical.update_layout(height=600)
        visualizations['clinical_significance'] = fig_clinical
        fig_clinical.show()
    
    print("✅ All visualizations created successfully!")
    return visualizations

In [21]:

# Create comprehensive visualizations
visualizations = create_hypothesis_testing_visualizations(categorical_results, continuous_results, correction_results)

📊 CREATING HYPOTHESIS TESTING VISUALIZATIONS
Creating p-values visualization...


Creating effect sizes comparison...


Creating multiple testing correction visualization...


Creating clinical significance assessment...


✅ All visualizations created successfully!


## 📋 Final Results Summary and Clinical Interpretation

Comprehensive summary of all hypothesis testing results with clinical recommendations and evidence-based conclusions.

In [22]:
def generate_comprehensive_hypothesis_testing_report(categorical_results, continuous_results, correction_results):
    """
    Generate comprehensive final report with clinical interpretation
    """
    print("="*100)
    print("📋 COMPREHENSIVE HYPOTHESIS TESTING REPORT")
    print("="*100)
    
    # Header information
    print(f"\nStroke Prediction Analysis - Statistical Hypothesis Testing")
    print(f"Analysis Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"Methodology: Rigorous statistical testing with medical domain validation")
    
    # Collect all significant findings
    strong_evidence = []
    moderate_evidence = []
    weak_evidence = []
    no_evidence = []
    
    total_tests = len(categorical_results) + len(continuous_results)
    
    print(f"\n🔬 ANALYSIS OVERVIEW:")
    print(f"   Total variables tested: {total_tests}")
    print(f"   Categorical variables: {len(categorical_results)}")
    print(f"   Continuous variables: {len(continuous_results)}")
    print(f"   Multiple testing correction: Applied (Bonferroni, FDR)")
    
    # Process categorical results
    print(f"\n" + "="*80)
    print("📊 CATEGORICAL VARIABLES ANALYSIS")
    print("="*80)
    
    for var, results in categorical_results.items():
        if 'tests' in results and 'chi_square' in results['tests']:
            test_result = results['tests']['chi_square']
            p_value = test_result['p_value']
            
            # Get correction results
            corrected = correction_results.get(var, {})
            bonferroni_sig = corrected.get('bonferroni_significant', False)
            fdr_sig = corrected.get('benjamini_hochberg_significant', False)
            
            # Get effect sizes
            effect_sizes = results.get('effect_sizes', {})
            odds_ratio = effect_sizes.get('odds_ratio', None)
            cramers_v = effect_sizes.get('cramers_v', 0)
            
            # Get clinical interpretation
            clinical = results.get('clinical_interpretation', {})
            clinical_significance = clinical.get('clinical_significance', 'No clinical significance')
            
            print(f"\n📋 {var.upper().replace('_', ' ')}")
            print(f"   Test: Chi-square test of independence")
            print(f"   Sample distribution:")
            
            # Print contingency table summary
            if 'contingency_table' in results:
                ct = results['contingency_table']
                print(f"   └── Contingency table available")
            
            print(f"   Statistical Results:")
            print(f"   ├── χ² = {test_result['statistic']:.4f}")
            print(f"   ├── p-value = {p_value:.4f}")
            print(f"   ├── Original significance: {'✅ Yes' if p_value < 0.05 else '❌ No'}")
            print(f"   ├── Bonferroni corrected: {'✅ Yes' if bonferroni_sig else '❌ No'}")
            print(f"   └── FDR corrected: {'✅ Yes' if fdr_sig else '❌ No'}")
            
            print(f"   Effect Sizes:")
            if odds_ratio is not None:
                ci_lower = effect_sizes.get('or_ci_lower', 'N/A')
                ci_upper = effect_sizes.get('or_ci_upper', 'N/A')
                print(f"   ├── Odds Ratio: {odds_ratio:.3f}")
                if ci_lower != 'N/A' and ci_upper != 'N/A':
                    print(f"   ├── 95% CI: [{ci_lower:.3f} - {ci_upper:.3f}]")
            print(f"   └── Cramér's V: {cramers_v:.3f}")
            
            print(f"   Clinical Assessment:")
            print(f"   ├── Clinical significance: {clinical_significance}")
            print(f"   └── Recommendation: {clinical.get('clinical_recommendation', 'No specific recommendation')}")
            
            # Classify evidence strength
            if bonferroni_sig and odds_ratio is not None:
                if odds_ratio > 2.0:
                    strong_evidence.append({
                        'variable': var,
                        'type': 'categorical',
                        'p_value': p_value,
                        'effect_size': odds_ratio,
                        'effect_type': 'Odds Ratio',
                        'clinical_significance': clinical_significance
                    })
                elif odds_ratio > 1.5:
                    moderate_evidence.append({
                        'variable': var,
                        'type': 'categorical',
                        'p_value': p_value,
                        'effect_size': odds_ratio,
                        'effect_type': 'Odds Ratio',
                        'clinical_significance': clinical_significance
                    })
                else:
                    weak_evidence.append({
                        'variable': var,
                        'type': 'categorical',
                        'p_value': p_value,
                        'effect_size': odds_ratio,
                        'effect_type': 'Odds Ratio',
                        'clinical_significance': clinical_significance
                    })
            elif fdr_sig:
                weak_evidence.append({
                    'variable': var,
                    'type': 'categorical',
                    'p_value': p_value,
                    'effect_size': odds_ratio or cramers_v,
                    'effect_type': 'Odds Ratio' if odds_ratio else 'Cramér\'s V',
                    'clinical_significance': clinical_significance
                })
            else:
                no_evidence.append({
                    'variable': var,
                    'type': 'categorical',
                    'p_value': p_value,
                    'effect_size': odds_ratio or cramers_v,
                    'effect_type': 'Odds Ratio' if odds_ratio else 'Cramér\'s V',
                    'clinical_significance': clinical_significance
                })
    
    # Process continuous results
    print(f"\n" + "="*80)
    print("📈 CONTINUOUS VARIABLES ANALYSIS")
    print("="*80)
    
    for var, results in continuous_results.items():
        # Get primary test result (prefer Mann-Whitney for robustness)
        test_result = None
        test_name = ""
        
        if 'mann_whitney' in results['tests']:
            test_result = results['tests']['mann_whitney']
            test_name = "Mann-Whitney U test"
        elif 't_test' in results['tests']:
            test_result = results['tests']['t_test']
            test_name = test_result.get('test_type', 't-test')
        
        if test_result is None:
            continue
            
        p_value = test_result['p_value']
        
        # Get correction results
        corrected = correction_results.get(var, {})
        bonferroni_sig = corrected.get('bonferroni_significant', False)
        fdr_sig = corrected.get('benjamini_hochberg_significant', False)
        
        # Get descriptive statistics
        desc_stats = results.get('descriptive_stats', {})
        no_stroke_stats = desc_stats.get('no_stroke', {})
        stroke_stats = desc_stats.get('stroke', {})
        
        # Get effect sizes
        effect_sizes = results.get('effect_sizes', {})
        cohens_d = effect_sizes.get('cohens_d', 0)
        mean_diff = effect_sizes.get('mean_difference', 0)
        point_biserial_r = effect_sizes.get('point_biserial_r', 0)
        
        # Get clinical interpretation
        clinical = results.get('clinical_interpretation', {})
        clinical_significance = clinical.get('clinical_significance', 'No clinical significance')
        
        print(f"\n📊 {var.upper().replace('_', ' ')}")
        print(f"   Test: {test_name}")
        print(f"   Sample sizes: No stroke = {no_stroke_stats.get('n', 'N/A'):,}, Stroke = {stroke_stats.get('n', 'N/A'):,}")
        
        print(f"   Descriptive Statistics:")
        print(f"   ├── No stroke: Mean = {no_stroke_stats.get('mean', 0):.3f}, SD = {no_stroke_stats.get('std', 0):.3f}")
        print(f"   ├── Stroke: Mean = {stroke_stats.get('mean', 0):.3f}, SD = {stroke_stats.get('std', 0):.3f}")
        print(f"   └── Mean difference: {mean_diff:.3f}")
        
        print(f"   Statistical Results:")
        if 'statistic' in test_result:
            print(f"   ├── Test statistic = {test_result['statistic']:.4f}")
        print(f"   ├── p-value = {p_value:.4f}")
        print(f"   ├── Original significance: {'✅ Yes' if p_value < 0.05 else '❌ No'}")
        print(f"   ├── Bonferroni corrected: {'✅ Yes' if bonferroni_sig else '❌ No'}")
        print(f"   └── FDR corrected: {'✅ Yes' if fdr_sig else '❌ No'}")
        
        print(f"   Effect Sizes:")
        print(f"   ├── Cohen's d: {cohens_d:.3f}")
        print(f"   ├── Point-biserial r: {point_biserial_r:.3f}")
        print(f"   └── Mean difference: {mean_diff:.3f}")
        
        print(f"   Clinical Assessment:")
        print(f"   ├── Clinical significance: {clinical_significance}")
        print(f"   └── Recommendation: {clinical.get('clinical_recommendation', 'No specific recommendation')}")
        
        # Classify evidence strength
        if bonferroni_sig:
            if abs(cohens_d) >= 0.8:
                strong_evidence.append({
                    'variable': var,
                    'type': 'continuous',
                    'p_value': p_value,
                    'effect_size': cohens_d,
                    'effect_type': 'Cohen\'s d',
                    'clinical_significance': clinical_significance,
                    'mean_difference': mean_diff
                })
            elif abs(cohens_d) >= 0.5:
                moderate_evidence.append({
                    'variable': var,
                    'type': 'continuous',
                    'p_value': p_value,
                    'effect_size': cohens_d,
                    'effect_type': 'Cohen\'s d',
                    'clinical_significance': clinical_significance,
                    'mean_difference': mean_diff
                })
            else:
                weak_evidence.append({
                    'variable': var,
                    'type': 'continuous',
                    'p_value': p_value,
                    'effect_size': cohens_d,
                    'effect_type': 'Cohen\'s d',
                    'clinical_significance': clinical_significance,
                    'mean_difference': mean_diff
                })
        elif fdr_sig:
            weak_evidence.append({
                'variable': var,
                'type': 'continuous',
                'p_value': p_value,
                'effect_size': cohens_d,
                'effect_type': 'Cohen\'s d',
                'clinical_significance': clinical_significance,
                'mean_difference': mean_diff
            })
        else:
            no_evidence.append({
                'variable': var,
                'type': 'continuous',
                'p_value': p_value,
                'effect_size': cohens_d,
                'effect_type': 'Cohen\'s d',
                'clinical_significance': clinical_significance,
                'mean_difference': mean_diff
            })
    
    # Evidence Summary
    print(f"\n" + "="*100)
    print("🎯 EVIDENCE SUMMARY AND CLINICAL RECOMMENDATIONS")
    print("="*100)
    
    print(f"\n🏆 STRONG EVIDENCE (Bonferroni significant + large effect size):")
    if strong_evidence:
        strong_evidence.sort(key=lambda x: x['p_value'])
        for i, evidence in enumerate(strong_evidence, 1):
            var = evidence['variable']
            effect_val = evidence['effect_size']
            effect_type = evidence['effect_type']
            p_val = evidence['p_value']
            clinical_sig = evidence['clinical_significance']
            
            print(f"   {i}. {var.replace('_', ' ').title()}")
            print(f"      ├── {effect_type}: {effect_val:.3f}")
            print(f"      ├── p-value: {p_val:.4f}")
            print(f"      ├── Clinical significance: {clinical_sig}")
            
            if evidence['type'] == 'continuous' and 'mean_difference' in evidence:
                print(f"      └── Mean difference: {evidence['mean_difference']:.3f}")
            else:
                print(f"      └── Strong risk factor for stroke")
    else:
        print("   ❌ No variables show strong evidence after conservative correction")
    
    print(f"\n🥈 MODERATE EVIDENCE (Bonferroni significant + moderate effect size):")
    if moderate_evidence:
        moderate_evidence.sort(key=lambda x: x['p_value'])
        for i, evidence in enumerate(moderate_evidence, 1):
            var = evidence['variable']
            effect_val = evidence['effect_size']
            effect_type = evidence['effect_type']
            p_val = evidence['p_value']
            clinical_sig = evidence['clinical_significance']
            
            print(f"   {i}. {var.replace('_', ' ').title()}")
            print(f"      ├── {effect_type}: {effect_val:.3f}")
            print(f"      ├── p-value: {p_val:.4f}")
            print(f"      ├── Clinical significance: {clinical_sig}")
            
            if evidence['type'] == 'continuous' and 'mean_difference' in evidence:
                print(f"      └── Mean difference: {evidence['mean_difference']:.3f}")
            else:
                print(f"      └── Moderate risk factor for stroke")
    else:
        print("   ❌ No variables show moderate evidence")
    
    print(f"\n🥉 WEAK EVIDENCE (FDR significant or small effect size):")
    if weak_evidence:
        weak_evidence.sort(key=lambda x: x['p_value'])
        for i, evidence in enumerate(weak_evidence, 1):
            var = evidence['variable']
            effect_val = evidence['effect_size']
            effect_type = evidence['effect_type']
            p_val = evidence['p_value']
            
            print(f"   {i}. {var.replace('_', ' ').title()}")
            print(f"      ├── {effect_type}: {effect_val:.3f}")
            print(f"      └── p-value: {p_val:.4f}")
    else:
        print("   ❌ No variables show weak evidence")
    
    print(f"\n❌ NO EVIDENCE (Non-significant after correction):")
    if no_evidence:
        no_evidence.sort(key=lambda x: x['p_value'])
        for i, evidence in enumerate(no_evidence, 1):
            var = evidence['variable']
            p_val = evidence['p_value']
            print(f"   {i}. {var.replace('_', ' ').title()} (p = {p_val:.4f})")
    else:
        print("   ✅ All variables showed some level of evidence")
    
    # Clinical Recommendations
    print(f"\n" + "="*100)
    print("🏥 CLINICAL RECOMMENDATIONS")
    print("="*100)
    
    print(f"\n💊 PRIMARY RISK FACTORS (Immediate clinical attention):")
    primary_factors = strong_evidence + moderate_evidence
    if primary_factors:
        for factor in primary_factors:
            var = factor['variable']
            clinical_sig = factor['clinical_significance']
            
            if var == 'age':
                print(f"   • AGE: Screen patients {factor.get('mean_difference', 0):.1f} years older for stroke risk")
            elif var == 'hypertension':
                print(f"   • HYPERTENSION: Critical modifiable risk factor (OR ≈ {factor['effect_size']:.2f})")
            elif var == 'heart_disease':
                print(f"   • HEART DISEASE: Major cardiovascular comorbidity (OR ≈ {factor['effect_size']:.2f})")
            elif 'glucose' in var.lower():
                print(f"   • GLUCOSE CONTROL: Monitor diabetic patients (Δ = {factor.get('mean_difference', 0):.1f} mg/dL)")
            elif 'bmi' in var.lower():
                print(f"   • BMI: Weight management important (Δ = {factor.get('mean_difference', 0):.1f} kg/m²)")
            else:
                print(f"   • {var.upper().replace('_', ' ')}: {clinical_sig}")
    else:
        print("   ⚠️  No primary risk factors identified - review inclusion criteria")
    
    print(f"\n⚠️  SECONDARY FACTORS (Monitor and consider):")
    if weak_evidence:
        for factor in weak_evidence:
            var = factor['variable']
            print(f"   • {var.replace('_', ' ').title()}: Consider in risk assessment")
    else:
        print("   ✅ No secondary factors requiring monitoring")
    
    print(f"\n🔍 FACTORS NOT ASSOCIATED WITH STROKE:")
    if no_evidence:
        for factor in no_evidence:
            var = factor['variable']
            print(f"   • {var.replace('_', ' ').title()}: No evidence of association")
    else:
        print("   ⚠️  All tested factors showed some association")
    
    # Statistical Quality Assessment
    print(f"\n" + "="*100)
    print("📊 STATISTICAL QUALITY ASSESSMENT")
    print("="*100)
    
    total_significant_original = len([1 for r in correction_results.values() if r.get('original_significant', False)])
    total_bonferroni = len([1 for r in correction_results.values() if r.get('bonferroni_significant', False)])
    total_fdr = len([1 for r in correction_results.values() if r.get('benjamini_hochberg_significant', False)])
    
    print(f"\n🔬 Multiple Testing Impact:")
    print(f"   ├── Original significant tests: {total_significant_original}/{total_tests} ({total_significant_original/total_tests*100:.1f}%)")
    print(f"   ├── After Bonferroni correction: {total_bonferroni}/{total_tests} ({total_bonferroni/total_tests*100:.1f}%)")
    print(f"   ├── After FDR correction: {total_fdr}/{total_tests} ({total_fdr/total_tests*100:.1f}%)")
    print(f"   └── Conservative estimate: {total_bonferroni} robust associations")
    
    print(f"\n📈 Effect Size Distribution:")
    large_effects = len(strong_evidence)
    medium_effects = len(moderate_evidence)
    small_effects = len(weak_evidence)
    
    print(f"   ├── Large effects (clinically important): {large_effects}")
    print(f"   ├── Medium effects (moderately important): {medium_effects}")
    print(f"   ├── Small effects (limited importance): {small_effects}")
    print(f"   └── No evidence: {len(no_evidence)}")
    
    # Final Conclusions
    print(f"\n" + "="*100)
    print("📋 FINAL CONCLUSIONS")
    print("="*100)
    
    print(f"\n🎯 KEY FINDINGS:")
    if len(strong_evidence + moderate_evidence) > 0:
        print(f"   ✅ {len(strong_evidence + moderate_evidence)} variables show robust evidence for stroke association")
        print(f"   ✅ Multiple testing corrections confirm {total_bonferroni} reliable associations")
        print(f"   ✅ Effect sizes support clinical relevance for primary risk factors")
    else:
        print(f"   ⚠️  Limited robust evidence after conservative corrections")
        print(f"   ⚠️  May indicate small effect sizes or underpowered analysis")
    
    print(f"\n🏥 CLINICAL IMPLICATIONS:")
    if strong_evidence:
        print(f"   • Prioritize screening and intervention for {len(strong_evidence)} primary risk factors")
        print(f"   • Implement evidence-based prevention strategies")
        print(f"   • Consider multifactorial risk assessment")
    
    if moderate_evidence:
        print(f"   • Monitor {len(moderate_evidence)} secondary risk factors")
        print(f"   • Include in comprehensive risk profiles")
    
    if no_evidence:
        print(f"   • {len(no_evidence)} factors show no association - may exclude from routine screening")
    
    print(f"\n📊 RESEARCH RECOMMENDATIONS:")
    print(f"   • Validate findings in independent cohorts")
    print(f"   • Consider interaction effects between risk factors")
    print(f"   • Develop multivariable prediction models")
    print(f"   • Assess temporal relationships and causality")
    
    print(f"\n⚠️  LIMITATIONS:")
    print(f"   • Cross-sectional analysis - no causal inference")
    print(f"   • Multiple testing may increase Type II error risk")
    print(f"   • Effect sizes may vary across populations")
    print(f"   • Clinical thresholds based on domain expertise")
    
    print(f"\n" + "="*100)
    print("✅ HYPOTHESIS TESTING ANALYSIS COMPLETE")
    print("="*100)
    
    # Return summary statistics
    summary_stats = {
        'total_tests': total_tests,
        'strong_evidence': len(strong_evidence),
        'moderate_evidence': len(moderate_evidence),
        'weak_evidence': len(weak_evidence),
        'no_evidence': len(no_evidence),
        'bonferroni_significant': total_bonferroni,
        'fdr_significant': total_fdr,
        'primary_risk_factors': [e['variable'] for e in strong_evidence + moderate_evidence],
        'secondary_factors': [e['variable'] for e in weak_evidence],
        'non_significant_factors': [e['variable'] for e in no_evidence]
    }
    
    return summary_stats



In [24]:
# Example usage and execution
print("🚀 Generating comprehensive hypothesis testing report...")

# Note: This assumes the previous analysis has been completed and results are available
summary_stats = generate_comprehensive_hypothesis_testing_report(
    categorical_results, 
    continuous_results, 
    correction_results
)

print("📋 Report generation function ready for execution!")
print("💡 Call the function with your analysis results to generate the full report.")

🚀 Generating comprehensive hypothesis testing report...
📋 COMPREHENSIVE HYPOTHESIS TESTING REPORT

Stroke Prediction Analysis - Statistical Hypothesis Testing
Analysis Date: 2025-06-19 12:01:52
Methodology: Rigorous statistical testing with medical domain validation

🔬 ANALYSIS OVERVIEW:
   Total variables tested: 11
   Categorical variables: 7
   Continuous variables: 4
   Multiple testing correction: Applied (Bonferroni, FDR)

📊 CATEGORICAL VARIABLES ANALYSIS

📋 GENDER
   Test: Chi-square test of independence
   Sample distribution:
   └── Contingency table available
   Statistical Results:
   ├── χ² = nan
   ├── p-value = nan
   ├── Original significance: ❌ No
   ├── Bonferroni corrected: ❌ No
   └── FDR corrected: ❌ No
   Effect Sizes:
   └── Cramér's V: nan
   Clinical Assessment:
   ├── Clinical significance: No clinical significance
   └── Recommendation: No evidence for clinical relevance

📋 HYPERTENSION
   Test: Chi-square test of independence
   Sample distribution:
   └── Co

In [25]:
def save_cleaned_dataset_with_documentation(df_cleaned, missing_strategy_log, output_dir="results"):
    """
    Save the cleaned dataset with comprehensive documentation
    """
    import os
    import json
    from pathlib import Path
    
    # Create output directory
    output_path = Path(output_dir)
    output_path.mkdir(exist_ok=True)
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    print("="*80)
    print("💾 SAVING CLEANED DATASET")
    print("="*80)
    
    # 1. Save the cleaned dataset in multiple formats
    print(f"\n📁 Saving cleaned dataset to: {output_path}")
    
    # CSV format (most compatible)
    csv_path = output_path / f"stroke_dataset_cleaned_{timestamp}.csv"
    df_cleaned.write_csv(csv_path)
    print(f"   ✅ CSV saved: {csv_path}")
    
    # Parquet format (efficient, preserves types)
    parquet_path = output_path / f"stroke_dataset_cleaned_{timestamp}.parquet"
    df_cleaned.write_parquet(parquet_path)
    print(f"   ✅ Parquet saved: {parquet_path}")
    
    # Excel format (for clinical users)
    excel_path = output_path / f"stroke_dataset_cleaned_{timestamp}.xlsx"
    df_cleaned.to_pandas().to_excel(excel_path, index=False, sheet_name="CleanedData")
    print(f"   ✅ Excel saved: {excel_path}")
    
    # 2. Create data cleaning documentation
    cleaning_doc = {
        "dataset_info": {
            "original_name": "healthcaredatasetstrokedata 1.csv",
            "cleaned_timestamp": datetime.now().isoformat(),
            "final_shape": {
                "rows": df_cleaned.shape[0],
                "columns": df_cleaned.shape[1]
            },
            "total_missing_values_removed": sum([log['missing_before'] for log in missing_strategy_log])
        },
        "cleaning_steps_applied": missing_strategy_log,
        "variable_info": {},
        "quality_metrics": {
            "completeness": "100% - all missing values handled",
            "consistency": "Validated data types and ranges",
            "accuracy": "Clinical domain validation applied"
        },
        "recommended_usage": {
            "modeling": "Ready for machine learning pipelines",
            "analysis": "Suitable for statistical hypothesis testing",
            "clinical": "Appropriate for clinical decision support research"
        }
    }
    
    # Add variable information
    for col in df_cleaned.columns:
        col_info = {
            "data_type": str(df_cleaned[col].dtype),
            "unique_values": df_cleaned[col].n_unique(),
            "missing_values": df_cleaned[col].null_count(),
        }
        
        # Add descriptive statistics for numeric columns
        if df_cleaned[col].dtype in [pl.Int8, pl.Int16, pl.Int32, pl.Int64, 
                                   pl.Float32, pl.Float64]:
            col_info.update({
                "mean": df_cleaned[col].mean(),
                "std": df_cleaned[col].std(),
                "min": df_cleaned[col].min(),
                "max": df_cleaned[col].max(),
                "median": df_cleaned[col].median()
            })
        else:
            # Categorical variables
            value_counts = df_cleaned[col].value_counts().to_pandas()
            col_info["value_counts"] = value_counts.to_dict()
        
        cleaning_doc["variable_info"][col] = col_info
    
    # Save documentation
    doc_path = output_path / f"cleaning_documentation_{timestamp}.json"
    with open(doc_path, 'w') as f:
        json.dump(cleaning_doc, f, indent=2, default=str)
    print(f"   ✅ Documentation saved: {doc_path}")
    
    # 3. Create human-readable cleaning report
    report_path = output_path / f"cleaning_report_{timestamp}.md"
    
    report_content = f"""# Stroke Dataset - Data Cleaning Report

## Overview
- **Original Dataset**: healthcaredatasetstrokedata 1.csv
- **Cleaning Date**: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
- **Final Shape**: {df_cleaned.shape[0]:,} rows × {df_cleaned.shape[1]} columns
- **Quality Score**: 100% (all missing values handled)

## Cleaning Steps Applied

"""
    
    for i, step in enumerate(missing_strategy_log, 1):
        report_content += f"""### {i}. {step['variable'].title()} - {step['method']}
- **Variable**: `{step['variable']}`
- **Method**: {step['method']}
- **Missing values handled**: {step['missing_before']}
- **Rationale**: {step['rationale']}

"""
    
    report_content += f"""## Variable Summary

| Variable | Type | Unique Values | Mean/Mode | Std/Categories |
|----------|------|---------------|-----------|----------------|
"""
    
    for col in df_cleaned.columns:
        dtype = str(df_cleaned[col].dtype)
        unique_count = df_cleaned[col].n_unique()
        
        if df_cleaned[col].dtype in [pl.Int8, pl.Int16, pl.Int32, pl.Int64, 
                                   pl.Float32, pl.Float64]:
            mean_val = f"{df_cleaned[col].mean():.3f}"
            std_val = f"{df_cleaned[col].std():.3f}"
        else:
            # For categorical, show most common value
            mode_val = df_cleaned[col].mode().first()
            mean_val = f"'{mode_val}'"
            categories = df_cleaned[col].unique().to_list()[:3]  # First 3 categories
            std_val = f"{len(categories)}+ categories"
        
        report_content += f"| {col} | {dtype} | {unique_count} | {mean_val} | {std_val} |\n"
    
    report_content += f"""
## Quality Assurance

### ✅ Completeness
- All missing values have been appropriately handled
- No null values remain in the dataset
- Imputation methods preserve data integrity

### ✅ Consistency  
- Data types validated and standardized
- Categorical values checked for consistency
- Numeric ranges validated against clinical norms

### ✅ Clinical Validation
- BMI values within realistic ranges (15-50 kg/m²)
- Age values appropriate for stroke analysis (18-95 years)
- Glucose levels within expected medical ranges
- Smoking status categories clinically meaningful

## Usage Recommendations

### 🤖 Machine Learning
The cleaned dataset is ready for:
- Feature engineering
- Model training and validation
- Cross-validation experiments
- Predictive modeling

### 📊 Statistical Analysis
Suitable for:
- Hypothesis testing (completed)
- Regression analysis
- Survival analysis
- Epidemiological studies

### 🏥 Clinical Research
Appropriate for:
- Risk factor identification
- Clinical decision support development
- Population health studies
- Healthcare policy research

## Files Generated

1. **stroke_dataset_cleaned_{timestamp}.csv** - Main cleaned dataset (CSV format)
2. **stroke_dataset_cleaned_{timestamp}.parquet** - Efficient binary format
3. **stroke_dataset_cleaned_{timestamp}.xlsx** - Excel format for clinical users
4. **cleaning_documentation_{timestamp}.json** - Machine-readable metadata
5. **cleaning_report_{timestamp}.md** - This human-readable report

## Citation

If using this cleaned dataset, please cite the cleaning methodology and acknowledge the data preprocessing steps applied.

---
*Generated by Stroke Prediction Analysis Pipeline - {datetime.now().strftime('%Y-%m-%d')}*
"""
    
    with open(report_path, 'w') as f:
        f.write(report_content)
    print(f"   ✅ Cleaning report saved: {report_path}")
    
    # 4. Create data dictionary
    dictionary_path = output_path / f"data_dictionary_{timestamp}.csv"
    
    data_dict_entries = []
    for col in df_cleaned.columns:
        entry = {
            'Variable': col,
            'Type': str(df_cleaned[col].dtype),
            'Description': get_variable_description(col),
            'Unique_Values': df_cleaned[col].n_unique(),
            'Missing_Values': df_cleaned[col].null_count(),
            'Clinical_Relevance': get_clinical_relevance(col)
        }
        
        if df_cleaned[col].dtype in [pl.Int8, pl.Int16, pl.Int32, pl.Int64, 
                                   pl.Float32, pl.Float64]:
            entry.update({
                'Min': df_cleaned[col].min(),
                'Max': df_cleaned[col].max(),
                'Mean': df_cleaned[col].mean(),
                'Std': df_cleaned[col].std()
            })
        
        data_dict_entries.append(entry)
    
    dict_df = pd.DataFrame(data_dict_entries)
    dict_df.to_csv(dictionary_path, index=False)
    print(f"   ✅ Data dictionary saved: {dictionary_path}")
    
    # 5. Summary statistics
    print(f"\n📊 DATASET SUMMARY:")
    print(f"   ├── Total records: {df_cleaned.shape[0]:,}")
    print(f"   ├── Total variables: {df_cleaned.shape[1]}")
    print(f"   ├── Missing values: 0 (100% complete)")
    print(f"   ├── Stroke cases: {df_cleaned['stroke'].sum():,} ({df_cleaned['stroke'].mean()*100:.1f}%)")
    print(f"   ├── Non-stroke cases: {(df_cleaned['stroke'] == 0).sum():,}")
    print(f"   └── Class balance ratio: {((df_cleaned['stroke'] == 0).sum() / df_cleaned['stroke'].sum()):.1f}:1")
    
    print(f"\n💾 FILES SAVED:")
    print(f"   ├── 📄 {csv_path.name}")
    print(f"   ├── 📦 {parquet_path.name}")
    print(f"   ├── 📊 {excel_path.name}")
    print(f"   ├── 📋 {doc_path.name}")
    print(f"   ├── 📝 {report_path.name}")
    print(f"   └── 📚 {dictionary_path.name}")
    
    return {
        'csv_path': csv_path,
        'parquet_path': parquet_path,
        'excel_path': excel_path,
        'documentation_path': doc_path,
        'report_path': report_path,
        'dictionary_path': dictionary_path,
        'cleaning_log': missing_strategy_log
    }

def get_variable_description(col_name):
    """Get human-readable description for each variable"""
    descriptions = {
        'id': 'Unique patient identifier',
        'gender': 'Patient gender (Male, Female, Other)',
        'age': 'Patient age in years',
        'hypertension': 'Hypertension status (0=No, 1=Yes)',
        'heart_disease': 'Heart disease status (0=No, 1=Yes)',
        'ever_married': 'Marital status (Yes/No)',
        'work_type': 'Type of work (Private, Self-employed, Govt_job, children, Never_worked)',
        'Residence_type': 'Residence type (Urban/Rural)',
        'avg_glucose_level': 'Average glucose level in mg/dL',
        'bmi': 'Body Mass Index in kg/m²',
        'smoking_status': 'Smoking status (never smoked, formerly smoked, smokes, Unknown)',
        'stroke': 'Stroke occurrence (0=No stroke, 1=Stroke)'
    }
    return descriptions.get(col_name, f'Variable: {col_name}')

def get_clinical_relevance(col_name):
    """Get clinical relevance for each variable"""
    relevance = {
        'id': 'Patient tracking and linkage',
        'gender': 'Gender differences in stroke epidemiology',
        'age': 'Strongest known stroke risk factor',
        'hypertension': 'Major modifiable cardiovascular risk factor',
        'heart_disease': 'Cardiovascular comorbidity increases stroke risk',
        'ever_married': 'Social determinant potentially affecting health outcomes',
        'work_type': 'Socioeconomic status and occupational factors',
        'Residence_type': 'Healthcare access and lifestyle differences',
        'avg_glucose_level': 'Diabetes/glucose control affects stroke risk',
        'bmi': 'Obesity relationship with cardiovascular health',
        'smoking_status': 'Major modifiable lifestyle risk factor',
        'stroke': 'Primary outcome of interest'
    }
    return relevance.get(col_name, 'Clinical relevance to be determined')

# Usage example with the pipeline
def save_cleaned_data_from_pipeline(pipeline):
    """
    Save cleaned dataset from the hypothesis testing pipeline
    """
    if pipeline.df_cleaned is not None and pipeline.missing_strategy_log:
        print("🚀 Saving cleaned dataset from pipeline...")
        
        saved_files = save_cleaned_dataset_with_documentation(
            pipeline.df_cleaned,
            pipeline.missing_strategy_log,
            output_dir="results/cleaned_data"
        )
        
        print(f"\n✅ Dataset successfully saved!")
        print(f"🎯 Primary file for analysis: {saved_files['csv_path']}")
        print(f"📊 For modeling: {saved_files['parquet_path']}")
        print(f"🏥 For clinical review: {saved_files['excel_path']}")
        
        return saved_files
    else:
        print("❌ No cleaned dataset available to save")
        return None

# Add this to the pipeline class
def add_save_functionality_to_pipeline():
    """
    Add save functionality to the existing pipeline
    """
    code_to_add = '''
    def save_cleaned_dataset(self, output_dir="results/cleaned_data"):
        """
        Save the cleaned dataset with comprehensive documentation
        """
        if self.df_cleaned is not None:
            return save_cleaned_dataset_with_documentation(
                self.df_cleaned,
                self.missing_strategy_log,
                output_dir
            )
        else:
            print("❌ No cleaned dataset available. Run data cleaning first.")
            return None
    '''
    
    print("📝 Add this method to your StrokeHypothesisTestingPipeline class:")
    print(code_to_add)

print("✅ Dataset saving functionality ready!")
print("💡 Use save_cleaned_data_from_pipeline(pipeline) after running the analysis")

✅ Dataset saving functionality ready!
💡 Use save_cleaned_data_from_pipeline(pipeline) after running the analysis


In [27]:
# After running the hypothesis testing pipeline
if pipeline.df_cleaned is not None:
    saved_files = save_cleaned_dataset_with_documentation(
        pipeline.df_cleaned,
        pipeline.missing_strategy_log,
        output_dir="results/cleaned_data"
    )
    
    print(f"✅ Cleaned dataset saved to: {saved_files['csv_path']}")

💾 SAVING CLEANED DATASET

📁 Saving cleaned dataset to: results/cleaned_data
   ✅ CSV saved: results/cleaned_data/stroke_dataset_cleaned_20250619_122557.csv
   ✅ Parquet saved: results/cleaned_data/stroke_dataset_cleaned_20250619_122557.parquet
   ✅ Excel saved: results/cleaned_data/stroke_dataset_cleaned_20250619_122557.xlsx
   ✅ Documentation saved: results/cleaned_data/cleaning_documentation_20250619_122557.json
   ✅ Cleaning report saved: results/cleaned_data/cleaning_report_20250619_122557.md
   ✅ Data dictionary saved: results/cleaned_data/data_dictionary_20250619_122557.csv

📊 DATASET SUMMARY:
   ├── Total records: 5,110
   ├── Total variables: 12
   ├── Missing values: 0 (100% complete)
   ├── Stroke cases: 249 (4.9%)
   ├── Non-stroke cases: 4,861
   └── Class balance ratio: 19.5:1

💾 FILES SAVED:
   ├── 📄 stroke_dataset_cleaned_20250619_122557.csv
   ├── 📦 stroke_dataset_cleaned_20250619_122557.parquet
   ├── 📊 stroke_dataset_cleaned_20250619_122557.xlsx
   ├── 📋 cleaning_docu